In [1]:
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import requests
import re
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [7]:
csv='input/DKSalaries (1).csv'

wk5 = pd.read_csv('input/qb/qb_2018_wk5.csv')
df1 = pd.read_csv('input/qb/qb_loc_cleaned.csv')

def_2018 = pd.read_csv('input/def/fantasy-football-points-allowed-defense-by-position.csv')


wk5_sal = pd.read_csv('input/qb/daily-fantasy-football-salary-and-projection-tool (2).csv')

In [8]:
# Parse upcoming week file by position

def clean_current(csv1):# Read in files

    df_upcoming = pd.read_csv(csv)
    def_2018 = pd.read_csv('input/def/fantasy-football-points-allowed-defense-by-position.csv')

    # Obtain home and away attributes

    df_upcoming['home'] = df_upcoming['Game Info'].apply(lambda x: (x.split('@')[1]).split()[0])
    df_upcoming['away'] = df_upcoming['Game Info'].apply(lambda x: (x.split('@')[0]))

    rb = []
    wr = []
    te = []
    qb= []
    for idx,pos in enumerate(df_upcoming['Position']):
        if pos == 'RB':
            rb.append(idx)
        elif pos =='WR':
            wr.append(idx)
        elif pos =='TE':
            te.append(idx)
        elif pos =='QB':
            qb.append(idx)


    # Sort current week dataframe

    qb_df = df_upcoming.iloc[qb]
    rb_df = df_upcoming.iloc[rb]
    wr_df = df_upcoming.iloc[wr]
    te_df = df_upcoming.iloc[te]
    
    return qb,df,wr_df,te_df

In [4]:
# A function designed to add the Draftkings fantasy football salary to every players statistics dataframe

def add_salary(df,df2):
    df = df.sort_values(['Week','PlayerID'])
    df = df.drop(['FantasyPointsPerGameDraftKings','Rank','Position'], axis=1)
    df.reset_index(inplace=True, drop=True)
    df2 = df2.sort_values(['Week','PlayerID'])
    df2 = df2.drop(['Rank','Position'], axis=1)
    df2.reset_index(inplace=True, drop=True)
    df['salary'] = df2['DraftKingsSalary']
    
    return df

    

In [26]:
# One hot encode all non numeric attributes


def one_hot(df):
    df.reset_index(inplace=True, drop=True)
    encoder = LabelEncoder()
    df['h/a_encoded'] = encoder.fit_transform(df['h/a'])
    df['surface_encoded'] = encoder.fit_transform(df['surface'])


    home =OneHotEncoder()
    surface = OneHotEncoder()

    X = home.fit_transform(df['h/a_encoded'].values.reshape(-1,1)).toarray()
    Xs = surface.fit_transform(df['surface_encoded'].values.reshape(-1,1)).toarray()

    dfOneHot = pd.DataFrame(X, columns = ["h/a_"+str(int(i)) for i in range(X.shape[1])])
    df = pd.concat([df, dfOneHot], axis=1)


    dfOneHot = pd.DataFrame(Xs, columns = ['surface_'+str(int(i)) for i in range(Xs.shape[1])])
    df = pd.concat([df, dfOneHot], axis=1)

    return df

In [44]:
"""This is a function designed to create a dataframe to hold the home and away teams in every NFL football game"""

def pfr_scraper(url, current_week):


# Set url for requests function

    page = requests.get(url)
# Use BeautifulSoup to scarpe data the data from Pro Footbal Reference.com 

    soup = BeautifulSoup(page.text,'html.parser')
    weeks = []  
    for tr in soup.find_all('tr'):
        weeks.append(tr.text)
        

# Remove any lines that dont describe weekly nfl game

    for row in weeks:
        try:
             weeks.remove( '\nWeek\nDay\nDate\nTime\nWinner/tie\n\nLoser/tie\n\nPtsW\nPtsL\nYdsW\nTOW\nYdsL\nTOL\n')
        except:
            None
    
# Split string a first non digit and use result for week varianble of each individual matchup

    week =[] 
    for row in weeks:
        if int(re.split(r'(\d+)', row)[1]) < current_week:
            week.append(int(re.split(r'(\d+)', row)[1]))
            
    return week, soup

In [80]:
# Prepare defense dataframe by sorting by relevant attribute reseting the indexand using rank as the reset dataframe index +1

def add_defensive_rank(df, df1, position):
    df1.sort_values(position_dict[position], inplace=True)
    df1.reset_index(inplace=True, drop=True)
    df1['Rank']= df1.index+1
    df1.index=df1['Team']

# Map defensive rnking to eack player in the statistical dataframe 

    df['def_rk'] = list(map(lambda x: df1.loc[x]['Rank'], df['Opponent']))
    
    return df


In [9]:
wk6 = clean_current(csv)
wk6.head()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,home,away
0,RB,Todd Gurley II (11421704),Todd Gurley II,11421704,RB/FLEX,10000,LAR@DEN 10/14/2018 04:05PM ET,LAR,29.30,DEN,LAR
1,WR,Antonio Brown (11421920),Antonio Brown,11421920,WR/FLEX,8700,PIT@CIN 10/14/2018 01:00PM ET,PIT,21.06,CIN,PIT
2,WR,Adam Thielen (11421922),Adam Thielen,11421922,WR/FLEX,8500,ARI@MIN 10/14/2018 01:00PM ET,MIN,27.78,MIN,ARI
3,RB,Le'Veon Bell (11421706),Le'Veon Bell,11421706,RB/FLEX,8500,PIT@CIN 10/14/2018 01:00PM ET,PIT,0.00,CIN,PIT
4,RB,Christian McCaffrey (11421708),Christian McCaffrey,11421708,RB/FLEX,8300,CAR@WAS 10/14/2018 01:00PM ET,CAR,22.52,WAS,CAR


In [85]:
df_upcoming['away'][:10]

0    LAR
1    PIT
2    ARI
3    PIT
4    CAR
5    LAC
6     TB
7    PIT
8     TB
9    PIT
Name: away, dtype: object

In [86]:
df_upcoming['year'] = 2018

In [7]:
rb_df.head()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,home,away,year
0,RB,Todd Gurley II (11421704),Todd Gurley II,11421704,RB/FLEX,10000,LAR@DEN 10/14/2018 04:05PM ET,LAR,29.30,DEN,LAR,2018
3,RB,Le'Veon Bell (11421706),Le'Veon Bell,11421706,RB/FLEX,8500,PIT@CIN 10/14/2018 01:00PM ET,PIT,0.00,CIN,PIT,2018
4,RB,Christian McCaffrey (11421708),Christian McCaffrey,11421708,RB/FLEX,8300,CAR@WAS 10/14/2018 01:00PM ET,CAR,22.52,WAS,CAR,2018
5,RB,Melvin Gordon III (11421710),Melvin Gordon III,11421710,RB/FLEX,8200,LAC@CLE 10/14/2018 01:00PM ET,LAC,26.70,CLE,LAC,2018
9,RB,James Conner (11421712),James Conner,11421712,RB/FLEX,7700,PIT@CIN 10/14/2018 01:00PM ET,PIT,23.82,CIN,PIT,2018


In [12]:
df1.head()

,games,year,Name,Team,Week,Opponent,att,comp%,yds,td,rat,def_rk,h/a_1,surface_0,salary,points
0,1,2015,AJ McCarron,CIN,12,LAR,1,100.0,3,0,79.17,10,1.0,0.0,6000,0.12
1,2,2015,AJ McCarron,CIN,13,CLE,3,66.7,19,0,84.03,28,0.0,1.0,5000,0.76
2,3,2015,AJ McCarron,CIN,14,PIT,32,68.8,280,2,90.62,24,1.0,0.0,5000,18.00
3,4,2015,AJ McCarron,CIN,15,SF,21,71.4,192,1,115.58,16,0.0,1.0,3000,11.88
4,5,2015,AJ McCarron,CIN,16,DEN,35,62.9,200,1,87.80,1,0.0,1.0,5100,13.10


In [13]:
df1.drop(['games'], axis=1, inplace=True)

In [14]:
wk5.head()

,Rank,PlayerID,Name,Team,Position,Week,Opponent,PassingCompletions,PassingAttempts,PassingCompletionPercentage,...,PassingYardsPerAttempt,PassingTouchdowns,PassingInterceptions,PassingRating,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,FantasyPointsPerGameDraftKings,FantasyPointsDraftKings
0,1,2593,Aaron Rodgers,GB,QB,5,DET,32,52,61.5,...,8.5,3,0,108.01,3,10,3.3,0,33.7,33.68
1,2,4314,Tom Brady,NE,QB,5,IND,34,44,77.3,...,7.8,3,2,102.56,3,-1,-0.3,1,32.5,32.54
2,3,7242,Drew Brees,NO,QB,5,WAS,26,29,89.7,...,12.5,3,0,153.23,2,1,0.5,0,30.7,30.72
3,4,16245,Blake Bortles,JAX,QB,5,KC,33,61,54.1,...,7.0,1,4,54.68,4,34,8.5,1,28.6,28.60
4,5,18973,CJ Beathard,SF,QB,5,ARI,34,54,63.0,...,6.5,2,2,78.40,2,7,3.5,1,27.7,27.66


In [16]:
df1.head()

,year,Name,Team,Week,Opponent,att,comp%,yds,td,rat,def_rk,h/a_1,surface_0,salary,points
0,2015,AJ McCarron,CIN,12,LAR,1,100.0,3,0,79.17,10,1.0,0.0,6000,0.12
1,2015,AJ McCarron,CIN,13,CLE,3,66.7,19,0,84.03,28,0.0,1.0,5000,0.76
2,2015,AJ McCarron,CIN,14,PIT,32,68.8,280,2,90.62,24,1.0,0.0,5000,18.00
3,2015,AJ McCarron,CIN,15,SF,21,71.4,192,1,115.58,16,0.0,1.0,3000,11.88
4,2015,AJ McCarron,CIN,16,DEN,35,62.9,200,1,87.80,1,0.0,1.0,5100,13.10


In [15]:
wk5.drop('PlayerID', axis=1, inplace=True)

In [18]:
wk5 = add_salary(wk5,wk5_sal)
wk5.head()

,Name,Team,Week,Opponent,PassingCompletions,PassingAttempts,PassingCompletionPercentage,PassingYards,PassingYardsPerAttempt,PassingTouchdowns,PassingInterceptions,PassingRating,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,FantasyPointsDraftKings,salary
0,Aaron Rodgers,GB,5,DET,32,52,61.5,442,8.5,3,0,108.01,3,10,3.3,0,33.68,4300
1,Alex Smith,WAS,5,NO,23,39,59.0,275,7.1,0,1,69.93,2,7,3.5,1,15.70,6300
2,Andrew Luck,IND,5,NE,38,59,64.4,365,6.2,3,2,84.36,0,0,0.0,0,27.60,5600
3,Andy Dalton,CIN,5,MIA,20,30,66.7,248,8.3,1,1,89.31,4,10,2.5,0,13.92,4000
4,Baker Mayfield,CLE,5,BAL,25,43,58.1,342,8.0,1,1,81.73,2,23,11.5,0,21.98,5800


In [3]:
""" 
A function to read and concatenate two csvs, then split resulting dataframe by year. Lastly
group smaller dataframes by name, week, and opponent
"""

def read_concat(csv1, csv2):
    dl = [pd.read_csv(csv1), pd.read_csv(csv2)]
    for item in dl:
        item.rename(columns={'PassingCompletions':'comp',
              'PassingAttempts':'att',
              'PassingCompletionPercentage':'comp%',
              'PassingYards':'yds',
              'PassingYardsPerAttempt':'yds/att',
              'PassingTouchdowns':'td',
              'PassingInterceptions':'int',
              'PassingRating':'rat',
              'RushingAttempts':'ru_att',
              'RushingYards':'ru_yds',
              'RushingYardsPerAttempt':'ru_yds/att',
              'RushingTouchdowns':'ru_td',
              'FantasyPointsDraftKings':'points'}, inplace=True)
       
        
        item.replace('null',0,inplace=True)
        item['salary']=item['salary'].apply(lambda x:int(x))
        

    
    

In [19]:
wk5.rename(columns={'PassingCompletions':'comp',
              'PassingAttempts':'att',
              'PassingCompletionPercentage':'comp%',
              'PassingYards':'yds',
              'PassingYardsPerAttempt':'yds/att',
              'PassingTouchdowns':'td',
              'PassingInterceptions':'int',
              'PassingRating':'rat',
              'RushingAttempts':'ru_att',
              'RushingYards':'ru_yds',
              'RushingYardsPerAttempt':'ru_yds/att',
              'RushingTouchdowns':'ru_td',
              'FantasyPointsDraftKings':'points'}, inplace=True)
       

In [20]:
wk5['year']=2018

In [24]:
wk5.drop(['comp','yds/att','int','ru_att','ru_yds','ru_yds/att','ru_td'], axis=1,inplace=True)

In [719]:
rb.columns

Index(['PlayerID', 'Name', 'Team', 'Week', 'Opponent', 'att', 'yds', 'yds/att',
       'td', 'tar', 'rec', 'rec_yds', 'rec_td', 'Fumbles', 'fum', 'points',
       'salary', 'h/a', 'surface', 'year', 'def_rk', 'h/a_encoded',
       'surface_encoded', 'h/a_0', 'h/a_1', 'surface_0', 'surface_1'],
      dtype='object')

In [627]:
rb_2018.columns

Index(['PlayerID', 'Name', 'Team', 'Week', 'Opponent', 'att', 'yds', 'yds/att',
       'td', 'tar', 'rec', 'rec_yds', 'rec_td', 'Fumbles', 'fum', 'points',
       'salary', 'def_rk', 'h/a'],
      dtype='object')

In [704]:
rb_2018.columns

Index(['year', 'PlayerID', 'Name', 'Team', 'Week', 'Opponent', 'att', 'yds',
       'yds/att', 'td', 'tar', 'rec', 'rec_yds', 'rec_td', 'fum', 'def_rk',
       'h/a_1', 'surface_0', 'salary', 'points'],
      dtype='object')

In [715]:
# Rename dicitonary for rb 
rb.rename(columns={'ReceivingTargets':'tar',
              'Receptions':'rec',
              'ReceptionPercentage':'rec%',
              'ReceivingYards':'rec_yds',
              'ReceivingYardsPerTarget':'yds/tar',
              'ReceivingYardsPerReception': 'yds/rec',
              'ReceivingTouchdowns':'rec_td',
              'RushingAttempts':'att',
              'RushingYards':'yds',
              'RushingYardsPerAttempt':'yds/att',
              'RushingTouchdowns':'td',
              'FumblesLost':'fum',  
              'FantasyPointsDraftKings':'points'},inplace=True)
       

In [716]:
cols = rb.columns.tolist()
cols = cols[-4:-3] +cols[:-6]+cols[-3:]+cols[-5:-4] +cols[-6:-5]
cols
#rb=rb[cols]
#rb.tail()

['h/a',
 'PlayerID',
 'Name',
 'Team',
 'Week',
 'Opponent',
 'att',
 'yds',
 'yds/att',
 'td',
 'tar',
 'rec',
 'rec_yds',
 'rec_td',
 'Fumbles',
 'fum',
 'surface',
 'year',
 'def_rk',
 'salary',
 'points']

In [708]:
temp = pd.concat([rb,rb_2018])
temp.head()

,year,PlayerID,Name,Team,Week,Opponent,att,yds,yds/att,td,tar,rec,rec_yds,rec_td,fum,def_rk,h/a_1,surface_0,salary,points
0,2016,1589,John Kuhn,NO,1,OAK,2,4,2.0,0,1,0,0,0,0,23,1.0,0.0,3000,0.4
1,2016,2262,Tim Hightower,NO,1,OAK,4,9,2.2,0,1,1,1,0,0,23,1.0,0.0,null,2.0
2,2016,2699,Matt Forte,NYJ,1,CIN,22,96,4.4,0,7,5,59,0,0,21,1.0,0.0,4300,20.5
3,2016,4640,Reggie Bush,BUF,1,BAL,3,-4,-1.3,0,1,0,0,0,0,13,0.0,1.0,6100,-0.4
4,2016,4738,Mike Tolbert,CAR,1,DEN,2,6,3.0,0,1,1,1,0,0,18,0.0,1.0,3000,1.7


In [606]:
rb_2018 = rb_2018[:11]

In [637]:
rb_2018['year']=2018

In [550]:
df1.rename(columns={'ru_att':'att',
                   'ru_yds':'yds',
                   'ru_yds/att':'yds/att',
                   'ru_td':'td',
                   'yds':'rec_yds',
                   'td':'rec_td'}, inplace=True)

In [551]:
df1.columns

Index(['year', 'PlayerID', 'Name', 'Team', 'Week', 'Opponent', 'att', 'yds',
       'yds/att', 'td', 'tar', 'rec', 'rec_yds', 'rec_td', 'fum', 'def_rk',
       'h/a_1', 'surface_0', 'salary', 'points'],
      dtype='object')

In [507]:
df.drop(['Fumbles'], axis=1, inplace=True)zz

In [547]:
df1 = df1[df1['year']!=2018]

In [612]:
df=add_salary(df_rb,df_sal)

In [580]:
df.head()

,PlayerID,Name,Team,Week,Opponent,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,ReceivingTargets,Receptions,ReceivingYards,ReceivingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,salary
0,4807,Adrian Peterson,WAS,1,ARI,26,96,3.7,1,3,2,70,0,1,1,23.6,4700
1,5451,Jonathan Stewart,NYG,1,JAX,2,8,4.0,0,0,0,0,0,0,0,0.8,4000
2,5820,Frank Gore,MIA,1,TEN,9,61,6.8,0,0,0,0,0,0,0,6.1,3700
3,6198,Darren Sproles,PHI,1,ATL,5,10,2.0,0,7,4,22,0,0,0,7.2,3800
4,11932,LeSean McCoy,BUF,1,BAL,7,22,3.1,0,3,1,-1,0,0,0,3.1,6000


In [295]:
df.head()

,PlayerID,Name,Team,Week,Opponent,ru_att,ru_yds,ru_yds/att,ru_td,tar,rec,yds,td,fum,points,salary
0,5820,Frank Gore,MIA,4,NE,11,41,3.7,0,3,2,16,1,0,13.7,3700
1,11932,LeSean McCoy,BUF,4,GB,5,24,4.8,0,6,3,13,0,0,6.7,3600
2,12239,Chris Ivory,BUF,4,GB,6,15,2.5,0,0,0,0,0,0,1.5,4400
3,12296,LeGarrette Blount,DET,4,DAL,7,12,1.7,0,0,0,0,0,0,1.2,5600
4,12386,Marshawn Lynch,OAK,4,CLE,20,130,6.5,0,5,3,27,0,0,21.7,4600


## Location dataframe

In [75]:
# A dictionary of NFL team names and their appropriate abbreviations 

team_dict = {'New England Patriots':'NE',
            'Green Bay Packers':'GB',
            'Tampa Bay Buccaneers':'TB',
            'Kansas City Chiefs':'KC',
            'San Diego Chargers':'SD',
            'St. Louis Rams':'STL',
            'New Orleans Saints':'NO',
            'New York Jets':'NYJ',
            'New York Giants':'NYG',
            'San Francisco 49ers':'SF',
            'Los Angeles Rams':'LAR',
            'Jacksonville Jaguars':'JAX',
            'Los Angeles Chargers':'LAC'}

In [27]:
# Set url for requests function

url='https://www.pro-football-reference.com/years/2018/games.htm'

In [45]:
week, soup = pfr_scraper(url,6)

IndexError: list index out of range

In [37]:
week

NameError: name 'week' is not defined

In [68]:
weeks.remove('\nWeek\nDay\nDate\nTime\nVisitor\n\nHome\n\nPtsW\nPtsL\nYdsW\nTOW\nYdsL\nTOL\n')

ValueError: list.remove(x): x not in list

In [56]:
weeks

['1ThuSeptember 68:20PMPhiladelphia EaglesAtlanta Falconsboxscore181223222991',
 '1SunSeptember 91:00PMPittsburgh Steelers@Cleveland Brownsboxscore212147263271',
 '1SunSeptember 91:00PMMinnesota VikingsSan Francisco 49ersboxscore241634313274',
 '1SunSeptember 91:00PMCincinnati Bengals@Indianapolis Coltsboxscore342333023802',
 '1SunSeptember 91:00PMJacksonville Jaguars@New York Giantsboxscore201530513242',
 '1SunSeptember 91:00PMBaltimore RavensBuffalo Billsboxscore47336911532',
 '1SunSeptember 91:00PMNew England PatriotsHouston Texansboxscore272038933252',
 '1SunSeptember 91:00PMTampa Bay Buccaneers@New Orleans Saintsboxscore484052904752',
 '1SunSeptember 91:00PMMiami DolphinsTennessee Titansboxscore272034223363',
 '1SunSeptember 94:05PMKansas City Chiefs@Los Angeles Chargersboxscore382836205412',
 '1SunSeptember 94:25PMCarolina PanthersDallas Cowboysboxscore16829312321',
 '1SunSeptember 94:25PMWashington Redskins@Arizona Cardinalsboxscore24642912132',
 '1SunSeptember 94:25PMDenver Bro

In [69]:
# Split string a first non digit and use result for week varianble of each individual matchup

week =[] 
for row in weeks:
    if int(re.split(r'(\d+)', row)[1]) < 6:
        week.append(int(re.split(r'(\d+)', row)[1]))
        
location = pd.DataFrame(columns = ['week','home','away','year'])

    # insert current weekly data into location dataframe

location ['week'] = week
    

In [70]:
page = requests.get(url)
soup = BeautifulSoup(page.text,'html.parser')
game =[]
for td in soup.find_all('td'):
    game.append(td.text)

# Create lists to establish the home and away teams in every game of an NFL season

home = []
away =[]
try:
    for idx,val in enumerate(game):
        if val == '':
            home.append(game[idx-1])
            away.append(game[idx+1])
        elif val =='@':
            home.append(game[idx+1])
            away.append(game[idx-1])
except:
    None

In [71]:
location['home']=home[0:len(week)]
location['away']=away[0:len(week)]
location['year']=2018
        
        

In [72]:
location.head()

,week,home,away,year
0,1,Philadelphia Eagles,Atlanta Falcons,2018
1,1,Cleveland Browns,Pittsburgh Steelers,2018
2,1,Minnesota Vikings,San Francisco 49ers,2018
3,1,Indianapolis Colts,Cincinnati Bengals,2018
4,1,New York Giants,Jacksonville Jaguars,2018


In [58]:
# Use beautiful to retrieve detail on each game in an NFL season

def loc_df(bso4):

    game =[]
    for td in bso4.find_all('td'):
        game.append(td.text)

    # Create lists to establish the home and away teams in every game of an NFL season

    home = []
    away =[]
    try:
        for idx,val in enumerate(game):
            if val == '':
                home.append(game[idx-1])
                away.append(game[idx+1])
            elif val =='@':
                home.append(game[idx+1])
                away.append(game[idx-1])
    except:
        None

    # Create a dataframe to hold the location of every NFL game by week


    # insert current weekly data into location dataframe

    

In [73]:
game =[]
for td in soup.find_all('td'):
    game.append(td.text)

# Create lists to establish the home and away teams in every game of an NFL season

home = []
away =[]
try:
    for idx,val in enumerate(game):
        if val == '':
            home.append(game[idx-1])
            away.append(game[idx+1])
        elif val =='@':
            home.append(game[idx+1])
            away.append(game[idx-1])
except:
    None

In [76]:
location= location.replace({"away": team_dict})
location= location.replace({"home": team_dict})

In [77]:
# Convert the NFL team names that are not in Team Dictiionary by capitalizing the first three character of the team name 

try:
    location['home'] = location['home'].apply(lambda x: x[0:3].upper())
    location['away'] = location['away'].apply(lambda x: x[0:3].upper())
except:
    None




In [78]:
location[:17]

,week,home,away,year
0,1,PHI,ATL,2018
1,1,CLE,PIT,2018
2,1,MIN,SF,2018
3,1,IND,CIN,2018
4,1,NYG,JAX,2018
5,1,BAL,BUF,2018
6,1,NE,HOU,2018
7,1,NO,TB,2018
8,1,MIA,TEN,2018
9,1,LAC,KC,2018


## Defensive Ranking

In [83]:
position_dict = {'qb':'DraftKingsQuarterbackFantasyPointsAllowedAverage',
                'wr':'DraftKingsWideReceiverFantasyPointsAllowedAverage',
                'rb':'DraftKingsRunningbackFantasyPointsAllowedAverage',
                'te':'DraftKingsTightEndFantasyPointsAllowedAverage'}

In [91]:
def_2018

,Rank,Name,Team,Position,Played,DraftKingsQuarterbackFantasyPointsAllowedAverage,DraftKingsRunningbackFantasyPointsAllowedAverage,DraftKingsWideReceiverFantasyPointsAllowedAverage,DraftKingsTightEndFantasyPointsAllowedAverage,DraftKingsKickerFantasyPointsAllowedAverage,DraftKingsFantasyPointsAllowedAverage
0,1,Jacksonville Jaguars,JAX,DST,5,13.45,18.90,27.26,10.60,3.80,74.01
1,2,Baltimore Ravens,BAL,DST,5,15.15,14.66,36.34,11.88,2.40,80.43
2,3,Chicago Bears,CHI,DST,4,16.84,12.10,41.88,12.42,1.00,84.24
3,4,Tennessee Titans,TEN,DST,5,17.24,16.78,39.10,9.30,4.00,86.42
4,5,Green Bay Packers,GB,DST,5,18.10,21.20,36.44,10.96,1.40,88.10
5,6,Seattle Seahawks,SEA,DST,5,16.84,23.00,39.70,9.30,2.80,91.64
6,7,Washington Redskins,WAS,DST,4,19.20,24.98,35.32,11.85,2.00,93.35
7,8,Dallas Cowboys,DAL,DST,5,19.43,23.50,34.52,12.18,3.80,93.43
8,9,Los Angeles Rams,LAR,DST,5,17.49,23.28,36.18,13.62,4.20,94.77
9,10,New York Giants,NYG,DST,6,18.90,25.43,33.23,10.07,7.50,95.14


In [92]:
df = add_defensive_rank(wk5,def_2018,'qb')

In [93]:
wk5.head()

,Name,Team,Week,Opponent,att,comp%,yds,td,rat,points,salary,year,def_rk
0,Aaron Rodgers,GB,5,DET,52,61.5,442,3,108.01,33.68,4300,2018,12
1,Alex Smith,WAS,5,NO,39,59.0,275,0,69.93,15.70,6300,2018,29
2,Andrew Luck,IND,5,NE,59,64.4,365,3,84.36,27.60,5600,2018,20
3,Andy Dalton,CIN,5,MIA,30,66.7,248,1,89.31,13.92,4000,2018,3
4,Baker Mayfield,CLE,5,BAL,43,58.1,342,1,81.73,21.98,5800,2018,2


## Home or Away

In [78]:
def home_away(week_list,df):
    wk_lst =[]
    for val in week_list:
        if val not in wk_lst:
            wk_lst.append(val)

    wk_dict ={}
    for wk in wk_lst:
        wk_dict['week{}'.format(wk)] = df[df['Week']=wk]



SyntaxError: invalid syntax (<ipython-input-78-1716d7a211c9>, line 9)

In [299]:
location[:17]

,week,home,away,year
0,1,PHI,ATL,2018
1,1,BAL,BUF,2018
2,1,MIA,TEN,2018
3,1,NYG,JAX,2018
4,1,MIN,SF,2018
5,1,NE,HOU,2018
6,1,CLE,PIT,2018
7,1,NO,TB,2018
8,1,IND,CIN,2018
9,1,LAC,KC,2018


In [615]:
week1 = df[df['Week']==1]
week2 = df[df['Week']==2]
week3 = df[df['Week']==3]
week4 = df[df['Week']==4]

In [616]:
week3.head()

,PlayerID,Name,Team,Week,Opponent,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,ReceivingTargets,Receptions,ReceivingYards,ReceivingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,salary,def_rk
231,4807,Adrian Peterson,WAS,3,GB,19,120,6.3,2,0,0,0,0,0,0,27.0,3000,9
232,5451,Jonathan Stewart,NYG,3,HOU,2,8,4.0,0,0,0,0,0,0,0,0.8,null,19
233,5820,Frank Gore,MIA,3,OAK,6,12,2.0,0,0,0,0,0,0,0,1.2,3000,24
234,12239,Chris Ivory,BUF,3,MIN,20,56,2.8,0,4,3,70,0,0,0,15.6,3700,8
235,12296,LeGarrette Blount,DET,3,NE,16,48,3.0,0,2,2,17,0,1,0,8.5,7300,13


In [95]:
loc_df = location[location['week']==5];home_list = []
for team in wk5['Team']:
    if team in list(loc_df['home']):
        home_list.append('home')
    else:
        home_list.append('away')

wk5['h/a'] = home_list

In [97]:
wk5[:32]

,Name,Team,Week,Opponent,att,comp%,yds,td,rat,points,salary,year,def_rk,h/a
0,Aaron Rodgers,GB,5,DET,52,61.5,442,3,108.01,33.68,4300,2018,12,away
1,Alex Smith,WAS,5,NO,39,59.0,275,0,69.93,15.70,6300,2018,29,away
2,Andrew Luck,IND,5,NE,59,64.4,365,3,84.36,27.60,5600,2018,20,away
3,Andy Dalton,CIN,5,MIA,30,66.7,248,1,89.31,13.92,4000,2018,3,home
4,Baker Mayfield,CLE,5,BAL,43,58.1,342,1,81.73,21.98,5800,2018,2,home
5,Ben Roethlisberger,PIT,5,ATL,29,65.5,250,3,112.72,22.60,5900,2018,30,home
6,Blake Bortles,JAX,5,KC,61,54.1,430,1,54.68,28.60,4000,2018,28,away
7,CJ Beathard,SF,5,ARI,54,63.0,349,2,78.40,27.66,5300,2018,7,home
8,Cam Newton,CAR,5,NYG,35,60.0,237,2,75.54,18.38,6900,2018,13,home
9,Carson Wentz,PHI,5,MIN,35,68.6,311,2,115.30,25.04,4500,2018,27,home


In [98]:
df1.columns

Index(['year', 'Name', 'Team', 'Week', 'Opponent', 'att', 'comp%', 'yds', 'td',
       'rat', 'def_rk', 'h/a_1', 'surface_0', 'salary', 'points'],
      dtype='object')

In [99]:
wk5.columns

Index(['Name', 'Team', 'Week', 'Opponent', 'att', 'comp%', 'yds', 'td', 'rat',
       'points', 'salary', 'year', 'def_rk', 'h/a'],
      dtype='object')

## Surface Type

In [101]:
surface_dict={'KC':'grass',
             'CAR':'grass',
              'WAS':'grass',
              'CLE':'grass',
              'MIA':'grass',
              'PIT':'grass',
              'GB':'grass',
              'SF':'grass',
              'PHI':'grass',
              'LAR':'grass',
              'BAL': 'grass',
              'TEN':'grass',
              'OAK':'grass',
              'LAC':'grass',
              'TB':'grass',
              'CHI':'grass',
              'DEN':'grass',
              'ARI':'grass',
              'DAL':'turf',
              'SEA':'turf',
              'DET':'turf',
              'ATL':'turf',
              'NE':'turf',
              'IND':'turf',
              'NO':'turf',
              'NYG':'turf',
              'NYJ':'turf',
              'HOU':'turf',
              'BUF':'turf',
              'CIN':'turf',
              'MIN':'turf',
              'JAX':'grass'
              
             }

In [102]:
df.index = df['Team']

# Add the type of field the player palyed on by mapping the field on which the game was played to the surface dictionary

surface = []
for idx, team in enumerate(wk5['Team']):
    if wk5.iloc[idx]['h/a']=='home':
        surface.append(surface_dict[team])
    else:
        surface.append(surface_dict[wk5.iloc[idx]['Opponent']])


wk5['surface'] = surface

In [103]:
wk5.head()

,Name,Team,Week,Opponent,att,comp%,yds,td,rat,points,salary,year,def_rk,h/a,h/a_encoded,surface
Team,,,,,,,,,,,,,,,,
GB,Aaron Rodgers,GB,5,DET,52,61.5,442,3,108.01,33.68,4300,2018,12,away,0,turf
WAS,Alex Smith,WAS,5,NO,39,59.0,275,0,69.93,15.70,6300,2018,29,away,0,turf
IND,Andrew Luck,IND,5,NE,59,64.4,365,3,84.36,27.60,5600,2018,20,away,0,turf
CIN,Andy Dalton,CIN,5,MIA,30,66.7,248,1,89.31,13.92,4000,2018,3,home,1,turf
CLE,Baker Mayfield,CLE,5,BAL,43,58.1,342,1,81.73,21.98,5800,2018,2,home,1,grass


## One Hot Encoding

In [104]:
wk5= one_hot(wk5)

In [106]:
df1.columns

Index(['year', 'Name', 'Team', 'Week', 'Opponent', 'att', 'comp%', 'yds', 'td',
       'rat', 'def_rk', 'h/a_1', 'surface_0', 'salary', 'points'],
      dtype='object')

In [105]:
wk5.columns

Index(['Name', 'Team', 'Week', 'Opponent', 'att', 'comp%', 'yds', 'td', 'rat',
       'points', 'salary', 'year', 'def_rk', 'h/a', 'h/a_encoded', 'surface',
       'surface_encoded', 'h/a_0', 'h/a_1', 'surface_0', 'surface_1'],
      dtype='object')

In [107]:
wk5.drop(['h/a','surface','h/a_encoded','surface_encoded','h/a_0','surface_1'], axis=1, inplace=True)

In [113]:
wk5.columns

Index(['Name', 'Team', 'Week', 'Opponent', 'att', 'comp%', 'yds', 'td', 'rat',
       'points', 'salary', 'year', 'def_rk', 'h/a_1', 'surface_0'],
      dtype='object')

In [119]:
cols = wk5.columns.tolist()
cols = cols[-4:-3] + cols[:9] + cols[-3:] + cols[-5:-4] + cols[-6:-5]
wk5= wk5[cols]
wk5.head()

,year,Name,Team,Week,Opponent,att,comp%,yds,td,rat,def_rk,h/a_1,surface_0,salary,points
0,2018,Aaron Rodgers,GB,5,DET,52,61.5,442,3,108.01,12,0.0,0.0,4300,33.68
1,2018,Alex Smith,WAS,5,NO,39,59.0,275,0,69.93,29,0.0,0.0,6300,15.70
2,2018,Andrew Luck,IND,5,NE,59,64.4,365,3,84.36,20,0.0,0.0,5600,27.60
3,2018,Andy Dalton,CIN,5,MIA,30,66.7,248,1,89.31,3,1.0,0.0,4000,13.92
4,2018,Baker Mayfield,CLE,5,BAL,43,58.1,342,1,81.73,2,1.0,1.0,5800,21.98


In [108]:
wk5.shape

(36, 15)

In [125]:
df1 = pd.concat([df1, wk5])
df1.tail()

,year,Name,Team,Week,Opponent,att,comp%,yds,td,rat,def_rk,h/a_1,surface_0,salary,points
31,2018,Ryan Tannehill,MIA,5,CIN,35,57.1,185,1,57.44,26,0.0,0.0,4000,10.10
32,2018,Sam Darnold,NYJ,5,DEN,22,45.5,198,3,98.11,17,1.0,0.0,5200,19.42
33,2018,Taysom Hill,NO,5,WAS,0,0.0,0,0,0.00,15,1.0,0.0,6300,8.30
34,2018,Teddy Bridgewater,NO,5,WAS,0,0.0,0,0,0.00,15,1.0,0.0,4500,-0.30
35,2018,Tom Brady,NE,5,IND,44,77.3,341,3,102.56,25,1.0,0.0,6500,32.54


In [122]:
temp.shape

(1829, 15)

In [124]:
temp

,year,Name,Team,Week,Opponent,att,comp%,yds,td,rat,def_rk,h/a_1,surface_0,salary,points
0,2015,AJ McCarron,CIN,12,LAR,1,100.0,3,0,79.17,10,1.0,0.0,6000,0.12
1,2015,AJ McCarron,CIN,13,CLE,3,66.7,19,0,84.03,28,0.0,1.0,5000,0.76
2,2015,AJ McCarron,CIN,14,PIT,32,68.8,280,2,90.62,24,1.0,0.0,5000,18.00
3,2015,AJ McCarron,CIN,15,SF,21,71.4,192,1,115.58,16,0.0,1.0,3000,11.88
4,2015,AJ McCarron,CIN,16,DEN,35,62.9,200,1,87.80,1,0.0,1.0,5100,13.10
5,2015,AJ McCarron,CIN,17,BAL,27,63.0,160,2,103.94,19,1.0,0.0,5300,14.60
6,2017,AJ McCarron,CIN,14,CHI,8,50.0,47,0,68.23,5,0.0,0.0,4400,1.88
7,2017,AJ McCarron,CIN,15,MIN,6,50.0,19,0,56.94,3,0.0,0.0,4800,0.76
8,2015,Aaron Rodgers,GB,1,CHI,23,78.3,189,3,140.49,25,0.0,1.0,8600,23.06
9,2015,Aaron Rodgers,GB,2,SEA,33,75.8,249,2,116.86,2,1.0,1.0,7100,22.26


In [126]:
df1.to_csv('input/qb/qb_loc_cleaned.csv', index=False)

In [200]:
df1.rename(columns={'PassingCompletions':'comp',
              'PassingAttempts':'att',
              'PassingCompletionPercentage':'comp%',
              'PassingYards':'yds',
              'PassingYardsPerAttempt':'yds/att',
              'PassingTouchdowns':'td',
              'PassingInterceptions':'int',
              'PassingRating':'rat',
              'RushingAttempts':'ru_att',
              'RushingYards':'ru_yds',
              'RushingYardsPerAttempt':'ru_yds/att',
              'RushingTouchdowns':'ru_td',
              'FantasyPointsDraftKings':'points'}, inplace=True)
       

In [734]:
cols = rb.columns.tolist()
cols = cols[-4:-3] +cols[:-6]+cols[-3:]+cols[-5:-4] +cols[-6:-5]
rb=rb[cols]
rb.head()

,year,PlayerID,Name,Team,Week,Opponent,att,yds,yds/att,td,tar,rec,rec_yds,rec_td,fum,def_rk,h/a_1,surface_0,salary,points
0,2015,733,Jerome Felton,BUF,1,IND,0,0,0.0,0,1,0,0,0,0,18,1.0,0.0,3000,0.0
1,2015,1589,John Kuhn,GB,1,CHI,0,0,0.0,0,0,0,0,0,0,22,0.0,1.0,3000,0.0
2,2015,2699,Matt Forte,CHI,1,GB,24,141,5.9,1,8,5,25,0,0,13,1.0,1.0,3000,30.6
3,2015,3382,Marcel Reece,OAK,1,CIN,0,0,0.0,0,4,3,26,2,0,15,1.0,1.0,7800,17.6
4,2015,4640,Reggie Bush,SF,1,MIN,2,8,4.0,0,1,0,0,0,0,9,1.0,1.0,3000,0.8


In [735]:
rb = pd.concat([rb,rb_2018])
rb.head()

,year,PlayerID,Name,Team,Week,Opponent,att,yds,yds/att,td,tar,rec,rec_yds,rec_td,fum,def_rk,h/a_1,surface_0,salary,points
0,2015,733,Jerome Felton,BUF,1,IND,0,0,0.0,0,1,0,0,0,0,18,1.0,0.0,3000,0.0
1,2015,1589,John Kuhn,GB,1,CHI,0,0,0.0,0,0,0,0,0,0,22,0.0,1.0,3000,0.0
2,2015,2699,Matt Forte,CHI,1,GB,24,141,5.9,1,8,5,25,0,0,13,1.0,1.0,3000,30.6
3,2015,3382,Marcel Reece,OAK,1,CIN,0,0,0.0,0,4,3,26,2,0,15,1.0,1.0,7800,17.6
4,2015,4640,Reggie Bush,SF,1,MIN,2,8,4.0,0,1,0,0,0,0,9,1.0,1.0,3000,0.8


In [411]:
df[df['salary']=='null']

,PlayerID,Name,Team,Week,Opponent,tar,rec,rec%,yds,td,...,ru_att,ru_yds,ru_yds/att,ru_td,fum,points,salary,def_rk,h/a_1,surface_0


In [736]:
rb.tail()

,year,PlayerID,Name,Team,Week,Opponent,att,yds,yds/att,td,tar,rec,rec_yds,rec_td,fum,def_rk,h/a_1,surface_0,salary,points
451,2018,20159,Mike Boone,MIN,4,LAR,0,0,0.0,0,1,1,1,0,0,11,0.0,1.0,4400,1.1
452,2018,20318,Gregory Howell Jr.,HOU,4,IND,0,0,0.0,0,0,0,0,0,0,17,0.0,0.0,3900,0.0
453,2018,20327,Roc Thomas,MIN,4,LAR,1,4,4.0,0,1,0,0,0,0,11,0.0,1.0,3800,0.4
454,2018,20327,Roc Thomas,MIN,4,LAR,1,4,4.0,0,1,0,0,0,0,11,0.0,1.0,3800,0.4
455,2018,20562,De'Lance Turner,BAL,4,PIT,1,4,4.0,0,1,1,10,0,0,4,0.0,1.0,3800,2.4


In [526]:
rb_2018.replace('null',0,inplace=True)

In [527]:
rb_2018['salary']=rb_2018['salary'].apply(lambda x:int(x))

In [555]:
df1.shape

(5700, 20)

In [528]:
rb_2018['year']= 2018

In [570]:
rb_2018.head()

,year,PlayerID,Name,Team,Week,Opponent,att,yds,yds,yds/att,...,rec,yds,yds,rec_td,fum,def_rk,h/a_1,surface_0,salary,points
0,2018,4807,Adrian Peterson,WAS,1,ARI,26,96,70,3.7,...,2,96,70,0,1,31,0.0,1.0,4700,23.6
1,2018,5451,Jonathan Stewart,NYG,1,JAX,2,8,0,4.0,...,0,8,0,0,0,7,1.0,0.0,4000,0.8
2,2018,5820,Frank Gore,MIA,1,TEN,9,61,0,6.8,...,0,61,0,0,0,5,1.0,1.0,3700,6.1
3,2018,6198,Darren Sproles,PHI,1,ATL,5,10,22,2.0,...,4,10,22,0,0,30,1.0,1.0,3800,7.2
4,2018,11932,LeSean McCoy,BUF,1,BAL,7,22,-1,3.1,...,1,22,-1,0,0,2,0.0,1.0,6000,3.1


In [571]:
df1.columns

Index(['year', 'PlayerID', 'Name', 'Team', 'Week', 'Opponent', 'att', 'yds',
       'yds/att', 'td', 'tar', 'rec', 'rec_yds', 'rec_td', 'fum', 'def_rk',
       'h/a_1', 'surface_0', 'salary', 'points'],
      dtype='object')

In [657]:
cols = rb_2018.columns.tolist()
cols = cols[-3:-2] +cols[0:-6] + cols[-4:-3] + cols[-2:]+ cols[-5:-4] +cols[-6:-5]
#rb_2018 = rb_2018[cols]
#rb_2018.tail()
rb_2018 = rb_2018[cols]
rb_2018.tail()

,year,PlayerID,Name,Team,Week,Opponent,att,yds,yds/att,td,tar,rec,rec_yds,rec_td,fum,def_rk,h/a_1,surface_0,salary,points
451,2018,20159,Mike Boone,MIN,4,LAR,0,0,0.0,0,1,1,1,0,0,11,0.0,1.0,4400,1.1
452,2018,20318,Gregory Howell Jr.,HOU,4,IND,0,0,0.0,0,0,0,0,0,0,17,0.0,0.0,3900,0.0
453,2018,20327,Roc Thomas,MIN,4,LAR,1,4,4.0,0,1,0,0,0,0,11,0.0,1.0,3800,0.4
454,2018,20327,Roc Thomas,MIN,4,LAR,1,4,4.0,0,1,0,0,0,0,11,0.0,1.0,3800,0.4
455,2018,20562,De'Lance Turner,BAL,4,PIT,1,4,4.0,0,1,1,10,0,0,4,0.0,1.0,3800,2.4


In [567]:
cols.shape

(5700, 20)

In [658]:
rb= pd.concat([df1, rb_2018])

In [331]:
cols = df1.columns.tolist()
cols = col[-3:-2] + cols[]

Index(['PlayerID', 'Name', 'Team', 'Week', 'Opponent', 'ru_att', 'ru_yds',
       'ru_yds/att', 'ru_td', 'tar', 'rec', 'yds', 'td', 'fum', 'points',
       'salary', 'def_rk', 'year', 'h/a_1', 'surface_0'],
      dtype='object')

In [436]:
df.head()

,year,PlayerID,Name,Team,Week,Opponent,tar,rec,rec%,yds,...,ru_att,ru_yds,ru_yds/att,ru_td,fum,def_rk,h/a_1,surface_0,salary,points
0,2018,3061,Marcedes Lewis,GB,4,BUF,0,0,0.0,0,...,0,0,0,0,0,21,1.0,1.0,2500,0.0
1,2018,7884,Antonio Gates,LAC,4,SF,5,2,40.0,27,...,0,0,0,0,0,27,1.0,1.0,2500,10.7
2,2018,8534,Jared Cook,OAK,4,CLE,13,8,61.5,110,...,0,0,0,0,0,22,1.0,1.0,2700,34.0
3,2018,10974,Rob Gronkowski,NE,4,MIA,7,4,57.1,44,...,0,0,0,0,0,14,1.0,0.0,4100,8.4
4,2018,11026,Jermaine Gresham,ARI,4,SEA,0,0,0.0,0,...,0,0,0,0,0,7,1.0,1.0,7000,0.0


In [370]:
cols = df.columns.tolist()
cols = cols[:1] + cols[-1:] +cols[1:-1]
df = df[cols]
df.head()

,year,PlayerID,Name,Team,Week,Opponent,ru_att,ru_yds,ru_yds/att,ru_td,tar,rec,yds,td,fum,def_rk,h/a_1,surface_0,salary,points
0,2018,18878,Frank Gore,MIA,4,NE,11,41,3.7,0,3,2,16,1,0,13,0.0,0.0,3000,13.7
1,2018,17923,LeSean McCoy,BUF,4,GB,5,24,4.8,0,6,3,13,0,0,9,0.0,1.0,3000,6.7
2,2018,16776,Chris Ivory,BUF,4,GB,6,15,2.5,0,0,0,0,0,0,9,0.0,1.0,3000,1.5
3,2018,19003,LeGarrette Blount,DET,4,DAL,7,12,1.7,0,0,0,0,0,0,16,0.0,0.0,7800,1.2
4,2018,16887,Marshawn Lynch,OAK,4,CLE,20,130,6.5,0,5,3,27,0,0,23,1.0,1.0,3000,21.7


In [449]:
df.columns

Index(['year', 'PlayerID', 'Name', 'Team', 'Week', 'Opponent', 'tar', 'rec',
       'rec%', 'yds', 'td', 'ReceivingLong', 'yds/tar', 'yds/rec', 'ru_att',
       'ru_yds', 'ru_yds/att', 'ru_td', 'fum', 'def_rk', 'h/a_1', 'surface_0',
       'salary', 'points'],
      dtype='object')

In [448]:
df1.columns

Index(['year', 'PlayerID', 'Name', 'Team', 'Week', 'Opponent', 'tar', 'rec',
       'rec%', 'yds', 'td', 'ReceivingLong', 'yds/tar', 'yds/rec', 'ru_att',
       'ru_yds', 'ru_yds/att', 'ru_td', 'fum', 'def_rk', 'h/a_1', 'surface_0',
       'salary', 'points'],
      dtype='object')

In [445]:
cols = df1.columns.tolist()
cols = cols[-3:-2]+cols[:-6] + cols[-4:-3] + cols[-2:] + cols[-5:-4] + cols[-6:-5]
df1 = df1[cols]

In [450]:
df1.head()

,year,PlayerID,Name,Team,Week,Opponent,tar,rec,rec%,yds,...,ru_att,ru_yds,ru_yds/att,ru_td,fum,def_rk,h/a_1,surface_0,salary,points
0,2015,3,Craig Stevens,TEN,1,TB,0,0,0.0,0,...,0,0,0.0,0,0,21,0.0,1.0,2500,0.0
1,2015,722,Jason Witten,DAL,1,NYG,9,8,88.9,60,...,0,0,0.0,0,0,31,1.0,0.0,4000,26.0
2,2015,852,Jacob Tamme,ATL,1,PHI,3,3,100.0,19,...,0,0,0.0,0,0,11,1.0,0.0,2600,4.9
3,2015,1181,Martellus Bennett,CHI,1,GB,7,5,71.4,55,...,0,0,0.0,0,0,14,1.0,1.0,4300,16.5
4,2015,1545,Anthony Fasano,TEN,1,TB,1,1,100.0,18,...,0,0,0.0,0,0,21,0.0,1.0,2500,2.8


In [451]:
te = pd.concat([df1,df])
te.tail()

,year,PlayerID,Name,Team,Week,Opponent,tar,rec,rec%,yds,...,ru_att,ru_yds,ru_yds/att,ru_td,fum,def_rk,h/a_1,surface_0,salary,points
79,2018,19943,Durham Smythe,MIA,4,NE,1,0,0.0,0,...,0,0,0.0,0,0,1,0.0,0.0,2500,0.0
80,2018,19947,Chris Herndon IV,NYJ,4,JAX,1,1,100.0,18,...,0,0,0.0,0,0,3,0.0,1.0,2600,2.8
81,2018,19950,Will Dissly,SEA,4,ARI,1,1,100.0,5,...,0,0,0.0,0,0,9,0.0,1.0,2500,1.5
82,2018,19988,Tyler Conklin,MIN,4,LAR,0,0,0.0,0,...,0,0,0.0,0,0,23,0.0,1.0,2500,0.0
83,2018,20020,Jordan Thomas,HOU,4,IND,0,0,0.0,0,...,0,0,0.0,0,0,24,0.0,0.0,0,0.0


In [659]:
rb.shape

(6945, 24)

In [660]:
df1.shape

(6489, 20)

In [739]:
rb.shape

(6290, 20)

In [740]:
rb['year'].unique()

array([2015, 2016, 2017, 2018], dtype=int64)

In [663]:
rb=rb[rb['year']!=2018]

In [741]:
rb.to_csv('input/rb/dk_stats_loc.csv', index=False)

## Year

In [124]:
df['year']=2018

In [119]:
qb_2018 = one_hot(qb_2018)

In [134]:
qb= pd.read_csv('input/qb/dk_stats_loc.csv')
#qb_2018 = pd.read_csv('input/qb/fantasy-football-leaders (1).csv')
#qb_2018_sal = pd.read_csv('input/qb/daily-fantasy-football-salary-and-projection-tool.csv')

In [70]:
df.head()

,Rank,PlayerID,Name,Team,Position,Week,Opponent,PassingCompletions,PassingAttempts,PassingCompletionPercentage,...,PassingYardsPerAttempt,PassingTouchdowns,PassingInterceptions,PassingRating,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,FantasyPointsPerGameDraftKings,FantasyPointsDraftKings
0,1,8283,Ryan Fitzpatrick,TB,QB,1,NO,21,28,75.0,...,14.9,4,0,156.25,12,36,3.0,1,45.3,45.28
1,2,7242,Drew Brees,NO,QB,3,ATL,39,49,79.6,...,8.1,3,0,120.75,3,7,2.3,2,43.5,43.54
2,3,732,Matt Ryan,ATL,QB,3,NO,26,35,74.3,...,10.7,5,0,148.10,4,12,3.0,0,43.2,43.16
3,4,17922,Jared Goff,LAR,QB,4,MIN,26,33,78.8,...,14.1,5,0,158.33,2,7,3.5,0,42.3,42.30
4,5,3807,Ben Roethlisberger,PIT,QB,2,KC,39,60,65.0,...,7.5,3,0,104.31,2,9,4.5,1,42.0,41.98


In [161]:
qb.columns

Index(['PlayerID', 'Name', 'Team', 'Week', 'Opponent', 'PassingCompletions',
       'PassingAttempts', 'PassingCompletionPercentage', 'PassingYards',
       'PassingYardsPerAttempt', 'PassingTouchdowns', 'PassingInterceptions',
       'PassingRating', 'RushingAttempts', 'RushingYards',
       'RushingYardsPerAttempt', 'RushingTouchdowns', 'salary', 'year',
       'def_rk', 'h/a', 'surface', 'FantasyPointsDraftKings'],
      dtype='object')

In [162]:
# Add salary to overall dataframe

qb_2018 = add_salary(qb_2018,qb_2018_sal)

In [163]:
qb_2018['year']=2018

In [164]:
qb_2018.head()

,PlayerID,Name,Team,Week,Opponent,PassingCompletions,PassingAttempts,PassingCompletionPercentage,PassingYards,PassingYardsPerAttempt,PassingTouchdowns,PassingInterceptions,PassingRating,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,FantasyPointsDraftKings,salary,year
0,611,Joe Flacco,BAL,1,BUF,25,34,73.5,236,6.9,3,0,121.69,2,3,1.5,0,21.74,4900,2018
1,732,Matt Ryan,ATL,1,PHI,21,43,48.8,251,5.8,0,1,57.41,2,8,4.0,0,9.84,6300,2018
2,2593,Aaron Rodgers,GB,1,CHI,20,30,66.7,286,9.5,3,0,130.69,1,15,15.0,0,24.94,4500,2018
3,3807,Ben Roethlisberger,PIT,1,CLE,23,41,56.1,335,8.2,1,3,60.52,3,16,5.3,0,17.00,7500,2018
4,4314,Tom Brady,NE,1,HOU,26,39,66.7,277,7.1,3,1,102.19,1,2,2.0,0,22.28,6600,2018


In [167]:
# Prepare defense dataframe by sorting by relevant attribute reseting the indexand using rank as the reset dataframe index +1

def_2018.sort_values('DraftKingsQuarterbackFantasyPointsAllowedAverage', inplace=True)
def_2018.reset_index(inplace=True)
def_2018['Rank']= def_2018.index+1
def_2018.index=def_2018['Team']



# Map defensive rnking to eack player in the statistical dataframe 

qb_2018['def_rk'] = list(map(lambda x: def_2018.loc[x]['Rank'], qb_2018['Opponent']))
qb_2018.head()

,PlayerID,Name,Team,Week,Opponent,PassingCompletions,PassingAttempts,PassingCompletionPercentage,PassingYards,PassingYardsPerAttempt,...,PassingInterceptions,PassingRating,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,FantasyPointsDraftKings,salary,year,def_rk
0,611,Joe Flacco,BAL,1,BUF,25,34,73.5,236,6.9,...,0,121.69,2,3,1.5,0,21.74,4900,2018,20
1,732,Matt Ryan,ATL,1,PHI,21,43,48.8,251,5.8,...,1,57.41,2,8,4.0,0,9.84,6300,2018,17
2,2593,Aaron Rodgers,GB,1,CHI,20,30,66.7,286,9.5,...,0,130.69,1,15,15.0,0,24.94,4500,2018,16
3,3807,Ben Roethlisberger,PIT,1,CLE,23,41,56.1,335,8.2,...,3,60.52,3,16,5.3,0,17.00,7500,2018,4
4,4314,Tom Brady,NE,1,HOU,26,39,66.7,277,7.1,...,1,102.19,1,2,2.0,0,22.28,6600,2018,12


In [169]:
qb=qb[qb['Week']!=4]

In [170]:
# add home or away, and surface attributes

week1  = qb_2018[qb_2018['Week']==1]
week2 = qb_2018[qb_2018['Week']==2]
week3 = qb_2018[qb_2018['Week']==3]


In [182]:
loc_df = location[location['week']==];home_list = []
for team in week3['Team']:
    if team in list(loc_df['home']):
        home_list.append('home')
    else:
        home_list.append('away')

week3['h/a'] = home_list

,PlayerID,Name,Team,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,ReceivingTouchdowns,...,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,year,salary,def_rk,h/a,surface
BUF,20250,Robert Foster,BUF,3,MIN,2,0,0.0,0,0,...,0.0,0,0,0,0.0,2018,0,22,home,turf
HOU,20295,Vyncint Smith,HOU,3,NYG,2,1,50.0,28,0,...,0.0,0,0,0,3.8,2018,0,8,home,turf
LAC,20310,JJ Jones,LAC,3,LAR,0,0,0.0,0,0,...,0.0,0,0,0,0.0,2018,0,12,home,grass
CLE,20477,Derrick Willies,CLE,3,NYJ,0,0,0.0,0,0,...,0.0,0,0,0,0.0,2018,0,17,home,grass
LAR,20679,Khadarel Hodge,LAR,3,LAC,0,0,0.0,0,0,...,0.0,0,0,0,0.0,2018,0,30,home,grass


In [68]:
qb_2018.head()

,FantasyPointsDraftKings,Name,Opponent,PassingAttempts,PassingCompletionPercentage,PassingCompletions,PassingInterceptions,PassingRating,PassingTouchdowns,PassingYards,...,RushingAttempts,RushingTouchdowns,RushingYards,RushingYardsPerAttempt,Team,Week,def_rk,h/a,salary,year
0,21.74,Joe Flacco,BUF,34,73.5,25,0,121.69,3,236,...,2,0,3,1.5,BAL,1,20,NaN,4900,2018
1,9.84,Matt Ryan,PHI,43,48.8,21,1,57.41,0,251,...,2,0,8,4.0,ATL,1,17,NaN,6300,2018
2,24.94,Aaron Rodgers,CHI,30,66.7,20,0,130.69,3,286,...,1,0,15,15.0,GB,1,16,NaN,4500,2018
3,17.00,Ben Roethlisberger,CLE,41,56.1,23,3,60.52,1,335,...,3,0,16,5.3,PIT,1,4,NaN,7500,2018
4,22.28,Tom Brady,HOU,39,66.7,26,1,102.19,3,277,...,1,0,2,2.0,NE,1,12,NaN,6600,2018


In [186]:
qb_2018 = pd.concat([qb_2018,week3])
qb_2018.head()

,PlayerID,Name,Team,Week,Opponent,PassingCompletions,PassingAttempts,PassingCompletionPercentage,PassingYards,PassingYardsPerAttempt,...,PassingRating,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,FantasyPointsDraftKings,salary,year,def_rk,h/a
0,611,Joe Flacco,BAL,1,BUF,25,34,73.5,236,6.9,...,121.69,2,3,1.5,0,21.74,4900,2018,20,home
1,732,Matt Ryan,ATL,1,PHI,21,43,48.8,251,5.8,...,57.41,2,8,4.0,0,9.84,6300,2018,17,away
2,2593,Aaron Rodgers,GB,1,CHI,20,30,66.7,286,9.5,...,130.69,1,15,15.0,0,24.94,4500,2018,16,home
3,3807,Ben Roethlisberger,PIT,1,CLE,23,41,56.1,335,8.2,...,60.52,3,16,5.3,0,17.00,7500,2018,4,away
4,4314,Tom Brady,NE,1,HOU,26,39,66.7,277,7.1,...,102.19,1,2,2.0,0,22.28,6600,2018,12,home


In [95]:
# Surface dictionary consisting of every team and the type of field they play on

surface_dict={'KC':'grass',
             'CAR':'grass',
              'WAS':'grass',
              'CLE':'grass',
              'MIA':'grass',
              'PIT':'grass',
              'GB':'grass',
              'SF':'grass',
              'PHI':'grass',
              'LAR':'grass',
              'BAL': 'grass',
              'TEN':'grass',
              'OAK':'grass',
              'LAC':'grass',
              'TB':'grass',
              'CHI':'grass',
              'DEN':'grass',
              'ARI':'grass',
              'DAL':'turf',
              'SEA':'turf',
              'DET':'turf',
              'ATL':'turf',
              'NE':'turf',
              'IND':'turf',
              'NO':'turf',
              'NYG':'turf',
              'NYJ':'turf',
              'HOU':'turf',
              'BUF':'turf',
              'CIN':'turf',
              'MIN':'turf',
              'JAX':'grass'
              
             }

In [188]:
qb_2018.index = qb_2018['Team']

# Add the type of field the player palyed on by mapping the field on which the game was played to the surface dictionary

surface = []
for idx, team in enumerate(qb_2018['Team']):
    if qb_2018.iloc[idx]['h/a']=='home':
        surface.append(surface_dict[team])
    else:
        surface.append(surface_dict[qb_2018.iloc[idx]['Opponent']])


qb_2018['surface'] = surface

In [106]:
cols = qb_2018.columns.tolist()
cols = cols[:17]+cols[18:]+cols[17:18]
qb_2018 = qb_2018[cols]
qb_2018.head()

,PlayerID,Name,Team,Week,Opponent,PassingCompletions,PassingAttempts,PassingCompletionPercentage,PassingYards,PassingYardsPerAttempt,...,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,salary,year,def_rk,h/a,surface,FantasyPointsDraftKings
Team,,,,,,,,,,,,,,,,,,,,,
BAL,611,Joe Flacco,BAL,1,BUF,25,34,73.5,236,6.9,...,2,3,1.5,0,4900,2018,20,home,grass,21.74
ATL,732,Matt Ryan,ATL,1,PHI,21,43,48.8,251,5.8,...,2,8,4.0,0,6300,2018,17,home,turf,9.84
GB,2593,Aaron Rodgers,GB,1,CHI,20,30,66.7,286,9.5,...,1,15,15.0,0,4500,2018,16,home,grass,24.94
PIT,3807,Ben Roethlisberger,PIT,1,CLE,23,41,56.1,335,8.2,...,3,16,5.3,0,7500,2018,4,home,grass,17.00
NE,4314,Tom Brady,NE,1,HOU,26,39,66.7,277,7.1,...,1,2,2.0,0,6600,2018,12,home,turf,22.28


In [98]:
qb=pd.read_csv('input/qb/dk_stats_loc.csv')

In [99]:
qb.head()

,PlayerID,Name,Team,Week,Opponent,PassingCompletions,PassingAttempts,PassingCompletionPercentage,PassingYards,PassingYardsPerAttempt,...,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,salary,year,def_rk,h/a,surface,FantasyPointsDraftKings
0,611,Joe Flacco,BAL,1,DEN,18,32,56.2,117,3.7,...,0,0,0.0,0,6700,2015,1,away,grass,2.68
1,732,Matt Ryan,ATL,1,PHI,23,34,67.6,298,8.8,...,5,7,1.4,0,7500,2015,30,home,turf,18.62
2,2428,Carson Palmer,ARI,1,NO,19,32,59.4,307,9.6,...,3,14,4.7,0,6500,2015,32,home,grass,28.68
3,2593,Aaron Rodgers,GB,1,CHI,18,23,78.3,189,8.2,...,8,35,4.4,0,8600,2015,25,away,grass,23.06
4,3807,Ben Roethlisberger,PIT,1,NE,26,38,68.4,351,9.2,...,0,0,0.0,0,7300,2015,15,away,turf,22.04


In [192]:
qb.head()

,FantasyPointsDraftKings,Name,Opponent,PassingAttempts,PassingCompletionPercentage,PassingCompletions,PassingInterceptions,PassingRating,PassingTouchdowns,PassingYards,...,RushingTouchdowns,RushingYards,RushingYardsPerAttempt,Team,Week,def_rk,h/a,salary,surface,year
0,2.68,Joe Flacco,DEN,32,56.2,18,2,38.15,0,117,...,0,0,0.0,BAL,1,1,away,6700,grass,2015
1,18.62,Matt Ryan,PHI,34,67.6,23,2,90.07,2,298,...,0,7,1.4,ATL,1,30,home,7500,turf,2015
2,28.68,Carson Palmer,NO,32,59.4,19,0,122.79,3,307,...,0,14,4.7,ARI,1,32,home,6500,grass,2015
3,23.06,Aaron Rodgers,CHI,23,78.3,18,0,140.49,3,189,...,0,35,4.4,GB,1,25,away,8600,grass,2015
4,22.04,Ben Roethlisberger,NE,38,68.4,26,1,95.39,1,351,...,0,0,0.0,PIT,1,15,away,7300,turf,2015


In [190]:
cols = qb_2018.columns.tolist()
cols = cols[]

Index(['PlayerID', 'Name', 'Team', 'Week', 'Opponent', 'PassingCompletions',
       'PassingAttempts', 'PassingCompletionPercentage', 'PassingYards',
       'PassingYardsPerAttempt', 'PassingTouchdowns', 'PassingInterceptions',
       'PassingRating', 'RushingAttempts', 'RushingYards',
       'RushingYardsPerAttempt', 'RushingTouchdowns',
       'FantasyPointsDraftKings', 'salary', 'year', 'def_rk', 'h/a',
       'surface'],
      dtype='object')

In [189]:
qb = pd.concat([qb,qb_2018])
qb.head()

,FantasyPointsDraftKings,Name,Opponent,PassingAttempts,PassingCompletionPercentage,PassingCompletions,PassingInterceptions,PassingRating,PassingTouchdowns,PassingYards,...,RushingTouchdowns,RushingYards,RushingYardsPerAttempt,Team,Week,def_rk,h/a,salary,surface,year
0,2.68,Joe Flacco,DEN,32,56.2,18,2,38.15,0,117,...,0,0,0.0,BAL,1,1,away,6700,grass,2015
1,18.62,Matt Ryan,PHI,34,67.6,23,2,90.07,2,298,...,0,7,1.4,ATL,1,30,home,7500,turf,2015
2,28.68,Carson Palmer,NO,32,59.4,19,0,122.79,3,307,...,0,14,4.7,ARI,1,32,home,6500,grass,2015
3,23.06,Aaron Rodgers,CHI,23,78.3,18,0,140.49,3,189,...,0,35,4.4,GB,1,25,away,8600,grass,2015
4,22.04,Ben Roethlisberger,NE,38,68.4,26,1,95.39,1,351,...,0,0,0.0,PIT,1,15,away,7300,turf,2015


In [194]:
qb=pd.read_csv('input/qb/dk_qb_loc.csv')

,PlayerID,Name,Team,Week,Opponent,PassingCompletions,PassingAttempts,PassingCompletionPercentage,PassingYards,PassingYardsPerAttempt,...,salary,year,def_rk,h/a,surface,h/a0,h/a1,surface_0,surface_1,FantasyPointsDraftKings
0,611,Joe Flacco,BAL,1,DEN,18,32,56.2,117,3.7,...,6700,2015,1,away,grass,1.0,0.0,1.0,0.0,2.68
1,732,Matt Ryan,ATL,1,PHI,23,34,67.6,298,8.8,...,7500,2015,30,home,turf,0.0,1.0,0.0,1.0,18.62
2,2428,Carson Palmer,ARI,1,NO,19,32,59.4,307,9.6,...,6500,2015,32,home,grass,0.0,1.0,1.0,0.0,28.68
3,2593,Aaron Rodgers,GB,1,CHI,18,23,78.3,189,8.2,...,8600,2015,25,away,grass,1.0,0.0,1.0,0.0,23.06
4,3807,Ben Roethlisberger,PIT,1,NE,26,38,68.4,351,9.2,...,7300,2015,15,away,turf,1.0,0.0,0.0,1.0,22.04


In [200]:
drop =[]
for week in enumerate(qb['Week']):
    if week==4:
        drop.append(idx)

In [203]:
qb.drop(drop, inplace=True)

In [204]:
qb.to_csv('input/qb/dk_stats_loc.csv')

## Wide Recievers

In [17]:
df1= read_concat('input/wr/2015wk1_2.csv','input/wr/2015wk3_4.csv')


#df2 = read_concat('input/wr/2015_sal_1_9.csv','input/wr/2015_sal_10_17.csv')

In [ ]:
df1 = pd.concat([df1,(pd.read_csv('input/wr/2015wk5_6.csv'))])
df1.reset_index(inplace=True, drop=True)

In [16]:
df1.shape

,Rank,PlayerID,Name,Team,Position,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,...,ReceivingYardsPerTarget,ReceivingYardsPerReception,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsPerGameDraftKings,FantasyPointsDraftKings
736,111,14368,Nick Toon,LAR,WR,17,SF,0,0,0.0,...,0.0,0.0,0,0,0.0,0,0,0,0.0,0.0
737,111,7857,Jerome Simpson,SF,WR,17,LAR,0,0,0.0,...,0.0,0.0,0,0,0.0,0,0,0,0.0,0.0
738,111,16480,Jeff Janis,GB,WR,17,MIN,0,0,0.0,...,0.0,0.0,0,0,0.0,0,0,0,0.0,0.0
739,111,16469,Jared Abbrederis,GB,WR,17,MIN,0,0,0.0,...,0.0,0.0,0,0,0.0,0,0,0,0.0,0.0
740,155,15150,Cordarrelle Patterson,MIN,WR,17,GB,0,0,0.0,...,0.0,0.0,0,0,0.0,0,1,1,-1.0,-1.0


In [257]:
data1 = pd.read_csv('input/te/daily-fantasy-football-salary-and-projection-tool.csv')
data2 = pd.read_csv('input/te/daily-fantasy-football-salary-and-projection-tool (1).csv')
data3 = pd.read_csv('input/te/daily-fantasy-football-salary-and-projection-tool (2).csv')
data4 = pd.read_csv('input/te/daily-fantasy-football-salary-and-projection-tool (3).csv')
data5 = pd.read_csv('input/te/daily-fantasy-football-salary-and-projection-tool (4).csv')
data6 = pd.read_csv('input/te/daily-fantasy-football-salary-and-projection-tool (5).csv')
#data7 = pd.read_csv('input/rb/daily-fantasy-football-salary-and-projection-tool (6).csv')
#data8 = pd.read_csv('input/rb/daily-fantasy-football-salary-and-projection-tool (7).csv')
#data9 = pd.read_csv('input/rb/daily-fantasy-football-salary-and-projection-tool (8).csv')


In [259]:
data6.head()

,Rank,PlayerID,Name,Team,Position,Week,Opponent,UpcomingOpponentRank,UpcomingOpponentPositionRank,DraftKingsSalary,FantasyPointsDraftKings
0,1,10974,Rob Gronkowski,NE,TE,16,NYJ,6,3,7600,19.4
1,2,1658,Greg Olsen,CAR,TE,16,ATL,11,28,7000,18.9
2,3,7175,Delanie Walker,TEN,TE,17,IND,26,19,5600,18.3
3,4,15100,Jordan Reed,WAS,TE,16,PHI,28,2,6500,18.0
4,5,10974,Rob Gronkowski,NE,TE,17,MIA,29,10,7400,17.5


In [198]:
#data1 = pd.read_csv('input/rb/fantasy-football-leaders.csv')
data2 = pd.read_csv('input/rb/fantasy-football-leaders (1).csv')
data3 = pd.read_csv('input/rb/fantasy-football-leaders (2).csv')
data4 = pd.read_csv('input/rb/fantasy-football-leaders (3).csv')
data5 = pd.read_csv('input/rb/fantasy-football-leaders (4).csv')
data6 = pd.read_csv('input/rb/fantasy-football-leaders (5).csv')
data7 = pd.read_csv('input/rb/fantasy-football-leaders (6).csv')
data8 = pd.read_csv('input/rb/fantasy-football-leaders (7).csv')
data9 = pd.read_csv('input/rb/fantasy-football-leaders (8).csv')
data10 = pd.read_csv('input/rb/fantasy-football-leaders (9).csv')

## Tight Ends

In [571]:
data1 = pd.read_csv('input/te/raw_2015_2016/daily-fantasy-football-salary-and-projection-tool.csv')
data2 = pd.read_csv('input/te/raw_2015_2016/daily-fantasy-football-salary-and-projection-tool (1).csv')
data3 = pd.read_csv('input/te/raw_2015_2016/daily-fantasy-football-salary-and-projection-tool (2).csv')
data4 = pd.read_csv('input/te/raw_2015_2016/daily-fantasy-football-salary-and-projection-tool (3).csv')
data5 = pd.read_csv('input/te/raw_2015_2016/daily-fantasy-football-salary-and-projection-tool (4).csv')
data6 = pd.read_csv('input/te/raw_2015_2016/daily-fantasy-football-salary-and-projection-tool (5).csv')

In [579]:
data2 = pd.read_csv('input/te/raw_2015_2016/fantasy-football-leaders (1).csv')
data3 = pd.read_csv('input/te/raw_2015_2016/fantasy-football-leaders (2).csv')
data4 = pd.read_csv('input/te/raw_2015_2016/fantasy-football-leaders (3).csv')
data5 = pd.read_csv('input/te/raw_2015_2016/fantasy-football-leaders (4).csv')
data6 = pd.read_csv('input/te/raw_2015_2016/fantasy-football-leaders (5).csv')
data7 = pd.read_csv('input/te/raw_2015_2016/fantasy-football-leaders (6).csv')

In [572]:
data5.head()

,Rank,PlayerID,Name,Team,Position,Week,Opponent,UpcomingOpponentRank,UpcomingOpponentPositionRank,DraftKingsSalary,FantasyPointsDraftKings
0,1,15048,Travis Kelce,KC,TE,13,ATL,30,28,4700,14.56
1,2,1658,Greg Olsen,CAR,TE,15,WAS,25,26,5000,14.39
2,3,11488,Jimmy Graham,SEA,TE,14,GB,17,23,5500,14.28
3,4,11488,Jimmy Graham,SEA,TE,13,CAR,25,31,5500,14.11
4,5,1658,Greg Olsen,CAR,TE,13,SEA,5,8,5100,13.95


In [504]:
te_2016.head()

,PlayerID,Name,Team,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,ReceivingTouchdowns,...,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,salary,def_rk,h/a,surface,year
0,722,Jason Witten,DAL,1,NYG,14,9,64.3,66,0,...,0,0,0,0,15.6,4100,20,away,turf,2016
1,852,Jacob Tamme,ATL,1,TB,8,6,75.0,51,0,...,0,0,0,0,11.1,2800,14,away,grass,2016
2,1181,Martellus Bennett,NE,1,ARI,5,3,60.0,14,0,...,0,0,0,0,4.4,3400,1,away,grass,2016
3,1545,Anthony Fasano,TEN,1,MIN,0,0,0.0,0,0,...,0,0,0,0,0.0,2500,11,away,turf,2016
4,1658,Greg Olsen,CAR,1,DEN,9,7,77.8,73,0,...,0,0,0,0,14.3,5100,19,away,grass,2016


In [584]:
te_2016 = pd.concat([te_2016,data7])
te_2016.tail()

,Rank,PlayerID,Name,Team,Position,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,...,ReceivingYardsPerTarget,ReceivingYardsPerReception,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsPerGameDraftKings,FantasyPointsDraftKings
162,103,15305,Demetrius Harris,KC,TE,17,LAC,1,0,0.0,...,0.0,0.0,0,0,0,0,0,0,0.0,0.0
163,103,16933,James O'Shaughnessy,KC,TE,17,LAC,0,0,0.0,...,0.0,0.0,0,0,0,0,0,0,0.0,0.0
164,103,11555,Jim Dray,SF,TE,17,SEA,0,0,0.0,...,0.0,0.0,0,0,0,0,0,0,0.0,0.0
165,103,9671,Matthew Mulligan,DET,TE,17,GB,0,0,0.0,...,0.0,0.0,0,0,0,0,0,0,0.0,0.0
166,103,11317,Clay Harbor,DET,TE,17,GB,0,0,0.0,...,0.0,0.0,0,0,0,0,0,0,0.0,0.0


In [578]:
te_2016_sal = te_2016

In [639]:
te_2016 = add_salary(te_2016,te_2016_sal)

te_2016.head()

,PlayerID,Name,Team,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,ReceivingTouchdowns,...,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,def_rk,h/a,surface,year,salary
0,722,Jason Witten,DAL,1,NYG,14,9,64.3,66,0,...,0,0,0,0,15.6,20,away,turf,2016,4100
1,852,Jacob Tamme,ATL,1,TB,8,6,75.0,51,0,...,0,0,0,0,11.1,14,away,grass,2016,2800
2,1181,Martellus Bennett,NE,1,ARI,5,3,60.0,14,0,...,0,0,0,0,4.4,1,away,grass,2016,3400
3,1545,Anthony Fasano,TEN,1,MIN,0,0,0.0,0,0,...,0,0,0,0,0.0,11,away,turf,2016,2500
4,1658,Greg Olsen,CAR,1,DEN,9,7,77.8,73,0,...,0,0,0,0,14.3,19,away,grass,2016,5100


In [586]:
def_2015.sort_values('DraftKingsTightEndFantasyPointsAllowedAverage', inplace=True)
def_2016.sort_values('DraftKingsTightEndFantasyPointsAllowedAverage', inplace=True)
def_2017.sort_values('DraftKingsTightEndFantasyPointsAllowedAverage', inplace=True)

In [587]:
def_2016.reset_index(inplace=True)


In [588]:
def_2016['Rank'] =def_2016.index+1

In [589]:
def_2016.index = def_2016['Team']

In [590]:
# Add defensive rankings to dataframe

#te_2015['def_rk'] = list(map(lambda x: def_2015.loc[x]['Rank'], te_2015['Opponent']))
te_2016['def_rk'] = list(map(lambda x: def_2016.loc[x]['Rank'], te_2016['Opponent']))
#te_2017['def_rk'] = list(map(lambda x: def_2017.loc[x]['Rank'], te_2017['Opponent']))

In [592]:
te_2016.shape

(1415, 24)

In [593]:
temp = te_2016

In [594]:
week1  = temp[temp['Week']==1]
week2 = temp[temp['Week']==2]
week3 = temp[temp['Week']==3]
week4 = temp[temp['Week']==4]
week5 = temp[temp['Week']==5]
week6 = temp[temp['Week']==6]
week7 = temp[temp['Week']==7]
week8 = temp[temp['Week']==8]
week9 = temp[temp['Week']==9]
week10 = temp[temp['Week']==10]
week11 = temp[temp['Week']==11]
week12 = temp[temp['Week']==12]
week13 = temp[temp['Week']==13]
week14 = temp[temp['Week']==14]
week15 = temp[temp['Week']==15]
week16 = temp[temp['Week']==16]
week17 = temp[temp['Week']==17]

In [595]:
loc_df = location[location['year']==2016]

In [627]:
loc_df[loc_df['week']==17]

home_list = []

for team in week17['Team']:
    if team in list(loc_df['home']):
        home_list.append('home')
    else:
        home_list.append('away')

week17['h/a'] = home_list




C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [628]:
te_2016 = pd.concat([te_2016,week17])
te_2016.tail()

,Rank,PlayerID,Name,Team,Position,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,...,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsPerGameDraftKings,FantasyPointsDraftKings,def_rk,h/a
162,103,15305,Demetrius Harris,KC,TE,17,LAC,1,0,0.0,...,0,0,0,0,0,0,0.0,0.0,12,away
163,103,16933,James O'Shaughnessy,KC,TE,17,LAC,0,0,0.0,...,0,0,0,0,0,0,0.0,0.0,12,away
164,103,11555,Jim Dray,SF,TE,17,SEA,0,0,0.0,...,0,0,0,0,0,0,0.0,0.0,4,away
165,103,9671,Matthew Mulligan,DET,TE,17,GB,0,0,0.0,...,0,0,0,0,0,0,0.0,0.0,21,away
166,103,11317,Clay Harbor,DET,TE,17,GB,0,0,0.0,...,0,0,0,0,0,0,0.0,0.0,21,away


In [629]:
surface = []
for idx, team in enumerate(te_2016['Team']):
    if te_2016.iloc[idx]['h/a']=='home':
        surface.append(surface_dict[team])
    else:
        surface.append(surface_dict[te_2016.iloc[idx]['Opponent']])


te_2016['surface'] = surface
te_2016.head()

,Rank,PlayerID,Name,Team,Position,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,...,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsPerGameDraftKings,FantasyPointsDraftKings,def_rk,h/a,surface
10,11,15602,Jack Doyle,IND,TE,1,DET,4,3,75.0,...,0,0,0,0,0,18.5,18.5,27,away,turf
12,13,13027,Julius Thomas,JAX,TE,1,GB,5,5,100.0,...,0,0,0,0,0,17.4,17.4,21,away,grass
13,14,13987,Dwayne Allen,IND,TE,1,DET,6,4,66.7,...,0,0,0,0,0,17.3,17.3,27,away,turf
17,18,16451,Eric Ebron,DET,TE,1,IND,5,5,100.0,...,0,0,0,0,0,15.6,15.6,23,away,turf
18,18,722,Jason Witten,DAL,TE,1,NYG,14,9,64.3,...,0,0,0,0,0,15.6,15.6,20,away,turf


In [630]:
te_2017['year']=2017
te_2016['year'] = 2016

In [637]:
te_2017.columns

Index(['PlayerID', 'Name', 'Team', 'Week', 'Opponent', 'ReceivingTargets',
       'Receptions', 'ReceptionPercentage', 'ReceivingYards',
       'ReceivingTouchdowns', 'ReceivingLong', 'ReceivingYardsPerTarget',
       'ReceivingYardsPerReception', 'RushingAttempts', 'RushingYards',
       'RushingYardsPerAttempt', 'RushingTouchdowns', 'Fumbles', 'FumblesLost',
       'FantasyPointsDraftKings', 'salary', 'def_rk', 'h/a', 'surface',
       'year'],
      dtype='object')

In [667]:
te_2017.shape

(1485, 25)

In [662]:
cols = te_2016.columns.tolist()
cols = cols[:-5] + cols[-1:] +cols[-5:-1]
te_2016 = te_2016[cols]

In [663]:
te_2016.shape

(1415, 25)

In [669]:
te=pd.concat([te,te_2017])
te.head()

,PlayerID,Name,Team,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,ReceivingTouchdowns,...,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,salary,def_rk,h/a,surface,year
0,3,Craig Stevens,TEN,1,TB,0,0,0.0,0,0,...,0.0,0,0,0,0.0,2500,21,away,grass,2015
1,722,Jason Witten,DAL,1,NYG,9,8,88.9,60,2,...,0.0,0,0,0,26.0,4000,31,away,turf,2015
2,852,Jacob Tamme,ATL,1,PHI,3,3,100.0,19,0,...,0.0,0,0,0,4.9,2600,11,away,grass,2015
3,1181,Martellus Bennett,CHI,1,GB,7,5,71.4,55,1,...,0.0,0,0,0,16.5,4300,14,away,grass,2015
4,1545,Anthony Fasano,TEN,1,TB,1,1,100.0,18,0,...,0.0,0,0,0,2.8,2500,21,away,grass,2015


In [670]:
te.tail()

,PlayerID,Name,Team,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,ReceivingTouchdowns,...,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,salary,def_rk,h/a,surface,year
1480,19304,Pharaoh Brown,OAK,17,LAC,0,0,0.0,0,0,...,0.0,0,0,0,0.0,2500,8,home,grass,2017
1481,19335,Antony Auclair,TB,17,NO,0,0,0.0,0,0,...,0.0,0,0,0,0.0,2500,2,away,turf,2017
1482,19390,Emanuel Byrd,GB,17,DET,2,2,100.0,31,0,...,0.0,0,0,0,5.1,3500,16,away,turf,2017
1483,19410,Ricky Seals-Jones,ARI,17,SEA,1,0,0.0,0,0,...,0.0,0,0,0,0.0,2500,9,home,grass,2017
1484,19579,Tyrone Swoopes,SEA,17,ARI,0,0,0.0,0,0,...,0.0,0,0,0,0.0,2500,17,home,turf,2017


In [671]:
te.to_csv('input/te/dk_stats_loc.csv', index=False)

## Running Backs

In [206]:
rb_2015= pd.concat([rb_2015,data10])
rb_2015.tail()

,Rank,PlayerID,Name,Team,Position,Week,Opponent,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,ReceivingTargets,Receptions,ReceivingYards,ReceivingTouchdowns,Fumbles,FumblesLost,FantasyPointsPerGameDraftKings,FantasyPointsDraftKings
115,88,13255,Matt Asiata,MIN,RB,17,GB,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0
116,88,15555,Zach Line,MIN,FB,17,GB,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0
117,118,15018,Denard Robinson,JAX,RB,17,HOU,5,9,1.8,0,1,0,0,0,1,1,-0.1,-0.1
118,119,17217,Raheem Mostert,CLE,RB,17,PIT,0,0,0.0,0,0,0,0,0,1,1,-1.0,-1.0
119,119,13852,Bobby Rainey,TB,RB,17,CAR,0,0,0.0,0,0,0,0,0,1,1,-1.0,-1.0


In [142]:
rb_2016 = pd.concat([rb_2016,data10])
rb_2016.tail()

,Rank,PlayerID,Name,Team,Position,Week,Opponent,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,ReceivingTargets,Receptions,ReceivingYards,ReceivingTouchdowns,Fumbles,FumblesLost,FantasyPointsPerGameDraftKings,FantasyPointsDraftKings
108,87,12323,Joique Bell,DET,RB,17,GB,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0
109,87,18152,Dwayne Washington,DET,RB,17,GB,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0
110,87,18452,Joe Kerridge,GB,FB,17,DET,1,0,0.0,0,0,0,0,0,0,0,0.0,0.0
111,112,18626,Brandon Burks,NYJ,RB,17,BUF,2,-4,-2.0,0,0,0,0,0,0,0,-0.4,-0.4
112,113,12985,DuJuan Harris,SF,RB,17,SEA,4,-1,-0.2,0,0,0,0,0,1,1,-1.1,-1.1


In [208]:
rb_2015 = add_salary(rb_2015,rb_2015_sal)

In [209]:
rb_2015.head()

,PlayerID,Name,Team,Week,Opponent,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,ReceivingTargets,Receptions,ReceivingYards,ReceivingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,salary
0,733,Jerome Felton,BUF,1,IND,0,0,0.0,0,1,0,0,0,0,0,0.0,3000
1,1589,John Kuhn,GB,1,CHI,0,0,0.0,0,0,0,0,0,0,0,0.0,3000
2,2699,Matt Forte,CHI,1,GB,24,141,5.9,1,8,5,25,0,0,0,30.6,3000
3,3382,Marcel Reece,OAK,1,CIN,0,0,0.0,0,4,3,26,2,0,0,17.6,7800
4,4640,Reggie Bush,SF,1,MIN,2,8,4.0,0,1,0,0,0,0,0,0.8,3000


In [145]:
def_2016.head()

,Rank,Name,Team,Position,Played,DraftKingsQuarterbackFantasyPointsAllowedAverage,DraftKingsRunningbackFantasyPointsAllowedAverage,DraftKingsWideReceiverFantasyPointsAllowedAverage,DraftKingsTightEndFantasyPointsAllowedAverage,DraftKingsKickerFantasyPointsAllowedAverage,DraftKingsFantasyPointsAllowedAverage
6,7,Arizona Cardinals,ARI,DST,16,17.59,18.91,40.28,6.30,0.0,83.07
21,22,Tennessee Titans,TEN,DST,16,20.68,19.09,44.60,11.48,0.0,95.85
17,18,Dallas Cowboys,DAL,DST,16,18.22,19.23,36.69,18.12,0.0,92.26
3,4,Seattle Seahawks,SEA,DST,16,15.73,19.99,37.16,9.02,0.0,81.90
13,14,Chicago Bears,CHI,DST,16,18.20,20.71,40.41,10.42,0.0,89.74


In [385]:
def_2015.sort_values('DraftKingsRunningbackFantasyPointsAllowedAverage', inplace=True)
def_2016.sort_values('DraftKingsRunningbackFantasyPointsAllowedAverage', inplace=True)
def_2017.sort_values('DraftKingsRunningbackFantasyPointsAllowedAverage', inplace=True)

In [386]:
def_2017.reset_index(inplace=True)
def_2017['Rank']= def_2016.index +1

In [210]:
rb_2015.index = rb_2015['Team']

In [370]:
rb_2015.head()

,PlayerID,Name,Team,Week,Opponent,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,ReceivingTargets,...,ReceivingYards,ReceivingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,salary,def_rk,h/a,surface,year
Team,,,,,,,,,,,,,,,,,,,,,
BUF,733,Jerome Felton,BUF,1,IND,0,0,0.0,0,1,...,0,0,0,0,0.0,3000,28,away,turf,2015
GB,1589,John Kuhn,GB,1,CHI,0,0,0.0,0,0,...,0,0,0,0,0.0,3000,15,away,grass,2015
CHI,2699,Matt Forte,CHI,1,GB,24,141,5.9,1,8,...,25,0,0,0,30.6,3000,5,away,grass,2015
OAK,3382,Marcel Reece,OAK,1,CIN,0,0,0.0,0,4,...,26,2,0,0,17.6,7800,9,away,turf,2015
SF,4640,Reggie Bush,SF,1,MIN,2,8,4.0,0,1,...,0,0,0,0,0.8,3000,11,away,turf,2015


In [387]:
def_2017.index=def_2017.Team

In [388]:
# Add defensive rankings to dataframe

#rb_2015['def_rk'] = list(map(lambda x: def_2015.loc[x]['Rank'], rb_2015['Opponent']))
#rb_2016['def_rk'] = list(map(lambda x: def_2016.loc[x]['Rank'], rb_2016['Opponent']))
rb_2017['def_rk'] = list(map(lambda x: def_2017.loc[x]['Rank'], rb_2017['Opponent']))

In [374]:
rb_2015.head()

,PlayerID,Name,Team,Week,Opponent,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,ReceivingTargets,...,ReceivingYards,ReceivingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,salary,def_rk,h/a,surface,year
Team,,,,,,,,,,,,,,,,,,,,,
BUF,733,Jerome Felton,BUF,1,IND,0,0,0.0,0,1,...,0,0,0,0,0.0,3000,18,away,turf,2015
GB,1589,John Kuhn,GB,1,CHI,0,0,0.0,0,0,...,0,0,0,0,0.0,3000,22,away,grass,2015
CHI,2699,Matt Forte,CHI,1,GB,24,141,5.9,1,8,...,25,0,0,0,30.6,3000,13,away,grass,2015
OAK,3382,Marcel Reece,OAK,1,CIN,0,0,0.0,0,4,...,26,2,0,0,17.6,7800,15,away,turf,2015
SF,4640,Reggie Bush,SF,1,MIN,2,8,4.0,0,1,...,0,0,0,0,0.8,3000,9,away,turf,2015


In [213]:
temp=rb_2015

In [214]:
week1  = temp[temp['Week']==1]
week2 = temp[temp['Week']==2]
week3 = temp[temp['Week']==3]
week4 = temp[temp['Week']==4]
week5 = temp[temp['Week']==5]
week6 = temp[temp['Week']==6]
week7 = temp[temp['Week']==7]
week8 = temp[temp['Week']==8]
week9 = temp[temp['Week']==9]
week10 = temp[temp['Week']==10]
week11 = temp[temp['Week']==11]
week12 = temp[temp['Week']==12]
week13 = temp[temp['Week']==13]
week14 = temp[temp['Week']==14]
week15 = temp[temp['Week']==15]
week16 = temp[temp['Week']==16]
week17 = temp[temp['Week']==17]

In [53]:
location = pd.read_csv('input/location.csv')

In [215]:
loc_df=location[location['year']==2015]

In [247]:
loc_df[loc_df['week']==17]

home_list = []
for team in week17['Team']:
    if team in list(loc_df['home']):
        home_list.append('home')
    else:
        home_list.append('away')

week17['h/a'] = home_list


week17.head()

C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,PlayerID,Name,Team,Week,Opponent,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,ReceivingTargets,Receptions,ReceivingYards,ReceivingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,salary,def_rk,h/a
Team,,,,,,,,,,,,,,,,,,,
BUF,733,Jerome Felton,BUF,17,NYJ,0,0,0.0,0,0,0,0,0,0,0,0.0,3500,6,away
WAS,957,Pierre Thomas,WAS,17,DAL,7,30,4.3,0,2,1,5,0,0,0,4.5,3600,8,away
GB,1589,John Kuhn,GB,17,MIN,1,3,3.0,0,2,2,14,0,0,0,3.7,3000,11,away
NE,1652,Steven Jackson,NE,17,MIA,14,35,2.5,1,1,1,20,0,0,0,12.5,3800,26,away
NO,2262,Tim Hightower,NO,17,ATL,16,66,4.1,1,5,5,41,0,0,0,21.7,3000,12,away


In [155]:
rb_2016 = pd.concat([rb_2015,week2]) 
rb_2016.head()

,PlayerID,Name,Team,Week,Opponent,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,ReceivingTargets,Receptions,ReceivingYards,ReceivingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,salary,def_rk,h/a
Team,,,,,,,,,,,,,,,,,,,
NO,1589,John Kuhn,NO,1,OAK,2,4,2.0,0,1,0,0,0,0,0,0.4,3000,21,away
NO,2262,Tim Hightower,NO,1,OAK,4,9,2.2,0,1,1,1,0,0,0,2.0,null,21,away
NYJ,2699,Matt Forte,NYJ,1,CIN,22,96,4.4,0,7,5,59,0,0,0,20.5,4300,11,away
BUF,4640,Reggie Bush,BUF,1,BAL,3,-4,-1.3,0,1,0,0,0,0,0,-0.4,6100,12,away
CAR,4738,Mike Tolbert,CAR,1,DEN,2,6,3.0,0,1,1,1,0,0,0,1.7,3000,1,away


In [248]:
rb_2015 = pd.concat([rb_2015, week17])
rb_2015.tail()

,PlayerID,Name,Team,Week,Opponent,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,ReceivingTargets,Receptions,ReceivingYards,ReceivingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,salary,def_rk,h/a
Team,,,,,,,,,,,,,,,,,,,
LAC,17120,Dreamius Smith,LAC,17,DEN,0,0,0.0,0,0,0,0,0,0,0,0.0,3000,2,away
HOU,17199,Akeem Hunt,HOU,17,JAX,3,28,9.3,0,3,1,27,0,0,0,6.5,4500,29,away
CLE,17217,Raheem Mostert,CLE,17,PIT,0,0,0.0,0,0,0,0,0,1,1,-1.0,3000,23,away
SF,17254,Jarryd Hayne,SF,17,LAR,0,0,0.0,0,0,0,0,0,0,0,0.0,3000,16,away
DAL,17360,Rod Smith,DAL,17,WAS,0,0,0.0,0,0,0,0,0,0,0,0.0,4600,27,away


In [613]:
rb_2016.head()

,PlayerID,Name,Team,Week,Opponent,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,ReceivingTargets,Receptions,ReceivingYards,ReceivingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,salary,def_rk,h/a
Team,,,,,,,,,,,,,,,,,,,
BUF,733,Jerome Felton,BUF,1,IND,0,0,0.0,0,1,0,0,0,0,0,0.0,3000,25,home
GB,1589,John Kuhn,GB,1,CHI,0,0,0.0,0,0,0,0,0,0,0,0.0,null,8,away
CHI,2699,Matt Forte,CHI,1,GB,24,141,5.9,1,8,5,25,0,0,0,30.6,4300,16,home
OAK,3382,Marcel Reece,OAK,1,CIN,0,0,0.0,0,4,3,26,2,0,0,17.6,6100,18,home
SF,4640,Reggie Bush,SF,1,MIN,2,8,4.0,0,1,0,0,0,0,0,0.8,3000,17,away


In [523]:
rb_2015.head()

,PlayerID,Name,Team,Week,Opponent,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,ReceivingTargets,Receptions,ReceivingYards,ReceivingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,salary,def_rk,h/a
Team,,,,,,,,,,,,,,,,,,,
BUF,733,Jerome Felton,BUF,1,IND,0,0,0.0,0,1,0,0,0,0,0,0.0,3000,28,home
GB,1589,John Kuhn,GB,1,CHI,0,0,0.0,0,0,0,0,0,0,0,0.0,3000,15,away
CHI,2699,Matt Forte,CHI,1,GB,24,141,5.9,1,8,5,25,0,0,0,30.6,3000,5,home
OAK,3382,Marcel Reece,OAK,1,CIN,0,0,0.0,0,4,3,26,2,0,0,17.6,7800,9,home
SF,4640,Reggie Bush,SF,1,MIN,2,8,4.0,0,1,0,0,0,0,0,0.8,3000,11,away


In [249]:
surface = []
for idx, team in enumerate(rb_2015['Team']):
    if rb_2015.iloc[idx]['h/a']=='home':
        surface.append(surface_dict[team])
    else:
        surface.append(surface_dict[rb_2015.iloc[idx]['Opponent']])


rb_2015['surface'] = surface
rb_2015.head()

,PlayerID,Name,Team,Week,Opponent,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,ReceivingTargets,Receptions,ReceivingYards,ReceivingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,salary,def_rk,h/a,surface
Team,,,,,,,,,,,,,,,,,,,,
BUF,733,Jerome Felton,BUF,1,IND,0,0,0.0,0,1,0,0,0,0,0,0.0,3000,28,away,turf
GB,1589,John Kuhn,GB,1,CHI,0,0,0.0,0,0,0,0,0,0,0,0.0,3000,15,away,grass
CHI,2699,Matt Forte,CHI,1,GB,24,141,5.9,1,8,5,25,0,0,0,30.6,3000,5,away,grass
OAK,3382,Marcel Reece,OAK,1,CIN,0,0,0.0,0,4,3,26,2,0,0,17.6,7800,9,away,turf
SF,4640,Reggie Bush,SF,1,MIN,2,8,4.0,0,1,0,0,0,0,0,0.8,3000,11,away,turf


In [250]:
rb_2017['year']=2017
rb_2016['year']=2016
rb_2015['year']=2015

In [389]:
rb = pd.concat([rb_2015,rb_2016])
rb.head()

,PlayerID,Name,Team,Week,Opponent,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,ReceivingTargets,...,ReceivingYards,ReceivingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,salary,def_rk,h/a,surface,year
Team,,,,,,,,,,,,,,,,,,,,,
BUF,733,Jerome Felton,BUF,1,IND,0,0,0.0,0,1,...,0,0,0,0,0.0,3000,18,away,turf,2015
GB,1589,John Kuhn,GB,1,CHI,0,0,0.0,0,0,...,0,0,0,0,0.0,3000,22,away,grass,2015
CHI,2699,Matt Forte,CHI,1,GB,24,141,5.9,1,8,...,25,0,0,0,30.6,3000,13,away,grass,2015
OAK,3382,Marcel Reece,OAK,1,CIN,0,0,0.0,0,4,...,26,2,0,0,17.6,7800,15,away,turf,2015
SF,4640,Reggie Bush,SF,1,MIN,2,8,4.0,0,1,...,0,0,0,0,0.8,3000,9,away,turf,2015


In [390]:
rb = pd.concat([rb,rb_2017])

In [391]:
rb.columns

Index(['PlayerID', 'Name', 'Team', 'Week', 'Opponent', 'RushingAttempts',
       'RushingYards', 'RushingYardsPerAttempt', 'RushingTouchdowns',
       'ReceivingTargets', 'Receptions', 'ReceivingYards',
       'ReceivingTouchdowns', 'Fumbles', 'FumblesLost',
       'FantasyPointsDraftKings', 'salary', 'def_rk', 'h/a', 'surface',
       'year'],
      dtype='object')

In [392]:
rb.to_csv('input/rb/dk_stats_loc.csv', index=False)

# Wide Receivers

In [281]:
wr_2016_df = pd.concat([wr_2016_df, data9])
wr_2016_df.head()

,Rank,PlayerID,Name,Team,Position,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,...,ReceivingYardsPerTarget,ReceivingYardsPerReception,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsPerGameDraftKings,FantasyPointsDraftKings
0,1,12845,AJ Green,CIN,WR,1,NYJ,13,12,92.3,...,13.8,15.0,0,0,0.0,0,0,0,39.0,39.0
1,2,16568,Brandin Cooks,NO,WR,1,OAK,9,6,66.7,...,15.9,23.8,1,11,11.0,0,0,0,36.4,36.4
2,3,16906,Stefon Diggs,MIN,WR,2,GB,11,9,81.8,...,16.5,20.2,0,0,0.0,0,0,0,36.2,36.2
3,4,11056,Antonio Brown,PIT,WR,1,WAS,11,8,72.7,...,11.5,15.8,0,0,0.0,0,0,0,35.6,35.6
4,5,16141,Willie Snead,NO,WR,1,OAK,9,9,100.0,...,19.1,19.1,0,0,0.0,0,1,0,35.2,35.2


In [77]:
wr_2017 = wr_2017[0:600]

In [44]:
df2.head()

,Rank,PlayerID,Name,Team,Position,Week,Opponent,UpcomingOpponentRank,UpcomingOpponentPositionRank,DraftKingsSalary,FantasyPointsDraftKings
0,1,4314,Tom Brady,NE,QB,9,WAS,16,15,8500,26.92
1,2,8244,Philip Rivers,LAC,QB,8,BAL,31,32,6600,26.72
2,3,4314,Tom Brady,NE,QB,5,DAL,18,10,7800,26.46
3,4,2428,Carson Palmer,ARI,QB,7,BAL,30,32,6700,26.30
4,5,13320,Cam Newton,CAR,QB,7,PHI,9,9,6600,26.22


In [111]:
wr_2015['year'] = 2015
wr_2016['year']=2016
wr_2017['year'] = 2017

In [40]:
def add_salary(df,df2):
    df = df.sort_values(['Week','PlayerID'])
    df = df.drop(['FantasyPointsPerGameDraftKings','Rank','Position'], axis=1)
    df.reset_index(inplace=True, drop=True)
    df2 = df2.sort_values(['Week','PlayerID'])
    df2 = df2.drop(['Rank','Position'], axis=1)
    df2.reset_index(inplace=True, drop=True)
    df['salary'] = df2['DraftKingsSalary']
    
    return df

    

In [331]:
wr_2016 = add_salary(wr_2016,wr_2016_sal)
wr_2016.head()

,PlayerID,Name,Team,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,ReceivingTouchdowns,...,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,def_rk,h/a,salary
0,2429,Andre Johnson,TEN,1,MIN,7,3,42.9,30,0,...,0,0,0.0,0,0,0,6.0,17,home,3900
1,3519,Vincent Jackson,TB,1,ATL,7,2,28.6,18,0,...,0,0,0.0,0,0,0,3.8,11,away,4400
2,3601,Eric Weems,ATL,1,TB,0,0,0.0,0,0,...,0,0,0.0,0,0,0,0.0,24,home,3000
3,3943,DeSean Jackson,WAS,1,PIT,10,6,60.0,102,0,...,0,0,0.0,0,0,0,19.2,22,home,6100
4,4358,Steve Smith,BAL,1,BUF,8,5,62.5,19,0,...,0,0,0.0,0,0,0,6.9,29,home,4900


In [383]:
cols =wr_2016.columns.tolist()
cols = cols[:-4] + cols[-1:] +cols[-3:-2] +cols[-4:-3] +cols[-2:-1]
df = wr_2016[cols]

In [385]:
df

,PlayerID,Name,Team,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,ReceivingTouchdowns,...,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,year,salary,def_rk,h/a
0,2429,Andre Johnson,TEN,1,MIN,7,3,42.9,30,0,...,0,0.0,0,0,0,6.0,2016,3900,17,home
1,3519,Vincent Jackson,TB,1,ATL,7,2,28.6,18,0,...,0,0.0,0,0,0,3.8,2016,4400,11,away
2,3601,Eric Weems,ATL,1,TB,0,0,0.0,0,0,...,0,0.0,0,0,0,0.0,2016,3000,24,home
3,3943,DeSean Jackson,WAS,1,PIT,10,6,60.0,102,0,...,0,0.0,0,0,0,19.2,2016,6100,22,home
4,4358,Steve Smith,BAL,1,BUF,8,5,62.5,19,0,...,0,0.0,0,0,0,6.9,2016,4900,29,home
5,4556,Jordy Nelson,GB,1,JAX,9,6,66.7,32,1,...,0,0.0,0,0,0,15.2,2016,7700,19,away
6,5571,Larry Fitzgerald,ARI,1,NE,10,8,80.0,81,2,...,0,0.0,0,0,0,28.1,2016,6300,26,home
7,6211,Harry Douglas,TEN,1,MIN,5,2,40.0,21,0,...,0,0.0,0,0,0,4.1,2016,3400,17,home
8,6767,Pierre Garcon,WAS,1,PIT,6,6,100.0,51,0,...,0,0.0,0,0,0,11.1,2016,4400,22,home
9,7082,Devin Hester,BAL,1,BUF,0,0,0.0,0,0,...,0,0.0,0,0,0,0.0,2016,null,29,home


In [400]:
wr = pd.read_csv('input/wr/dk_stats_loc.csv')

In [404]:
wr_2015 = wr[wr['year']==2015] 
wr_2016 = wr[wr['year']==2016]
wr_2017 = wr[wr['year']==2017]

In [371]:
wr_2016['year']=2016

In [386]:
wr_2016 = df

In [394]:
def_2015.sort_values('DraftKingsWideReceiverFantasyPointsAllowedAverage', inplace=True)
def_2016.sort_values('DraftKingsWideReceiverFantasyPointsAllowedAverage', inplace=True)
def_2017.sort_values('DraftKingsWideReceiverFantasyPointsAllowedAverage', inplace=True)

In [409]:
def_2017.reset_index(inplace=True)
def_2017['Rank'] = def_2017.index+1

In [410]:
#def_2015.index=def_2015['Team']
#def_2016.index=def_2016['Team']
def_2017.index=def_2017['Team']


In [285]:
wr_2016 = wr_2016_df

In [412]:
# Add defensive rankings to dataframe

#wr_2015['def_rk'] = list(map(lambda x: def_2015.loc[x]['Rank'], wr_2015['Opponent']))
#wr_2016['def_rk'] = list(map(lambda x: def_2016.loc[x]['Rank'], wr_2016['Opponent']))
wr_2017['def_rk'] = list(map(lambda x: def_2017.loc[x]['Rank'], wr_2017['Opponent']))

C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [287]:
wr_2016.head()

,Rank,PlayerID,Name,Team,Position,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,...,ReceivingYardsPerReception,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsPerGameDraftKings,FantasyPointsDraftKings,def_rk
0,1,12845,AJ Green,CIN,WR,1,NYJ,13,12,92.3,...,15.0,0,0,0.0,0,0,0,39.0,39.0,30
1,2,16568,Brandin Cooks,NO,WR,1,OAK,9,6,66.7,...,23.8,1,11,11.0,0,0,0,36.4,36.4,27
2,3,16906,Stefon Diggs,MIN,WR,2,GB,11,9,81.8,...,20.2,0,0,0.0,0,0,0,36.2,36.2,16
3,4,11056,Antonio Brown,PIT,WR,1,WAS,11,8,72.7,...,15.8,0,0,0.0,0,0,0,35.6,35.6,14
4,5,16141,Willie Snead,NO,WR,1,OAK,9,9,100.0,...,19.1,0,0,0.0,0,1,0,35.2,35.2,27


# Quarterbacks

In [74]:
df1= read_concat('input/qb/2016wk1_9.csv','input/qb/2016wk_10_17.csv')

df2 = read_concat('input/qb/2016_sal_wk1_9.csv','input/qb/2016_sal_wk10_17.csv')

df1= read_concat('input/qb/2017wk1_9.csv','input/qb/2017wk10_17.csv')

df2 = read_concat('input/qb/2017_sal_wk1_9.csv','input/qb/2017_sal_wk10_17.csv')

In [ ]:
# Add defensive rankings to dataframe

qb_2015['def_rk'] = list(map(lambda x: def_2015.loc[x]['Rank'], qb_2015['Opponent']))
qb_2016['def_rk'] = list(map(lambda x: def_2016.loc[x]['Rank'], qb_2016['Opponent']))
qb_2017['def_rk'] = list(map(lambda x: def_2017.loc[x]['Rank'], qb_2017['Opponent']))

In [76]:
qb.head()

,PlayerID,Name,Team,Week,Opponent,PassingCompletions,PassingAttempts,PassingCompletionPercentage,PassingYards,PassingYardsPerAttempt,PassingTouchdowns,PassingInterceptions,PassingRating,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,FantasyPointsDraftKings,salary
0,611,Joe Flacco,BAL,1,CIN,9,17,52.9,121,7.1,1,1,70.96,0,0,0.0,0,7.84,5500
1,732,Matt Ryan,ATL,1,CHI,21,30,70.0,321,10.7,1,0,116.11,3,11,3.7,0,20.94,6900
2,2428,Carson Palmer,ARI,1,DET,27,48,56.2,268,5.6,1,3,53.12,1,2,2.0,0,11.92,4300
3,2593,Aaron Rodgers,GB,1,SEA,28,42,66.7,311,7.4,1,1,86.51,7,21,3.0,0,20.54,6000
4,3807,Ben Roethlisberger,PIT,1,CLE,24,36,66.7,263,7.3,2,1,95.02,3,-8,-2.7,0,16.72,7000


In [421]:
qb=pd.read_csv('input/qb/dk_stats_loc.csv')

In [422]:
qb_2015 = qb[qb['year']==2015]
qb_2016 = qb[qb['year']==2016]
qb_2017 = qb[qb['year']==2017]

In [426]:
def_2015.sort_values('DraftKingsQuarterbackFantasyPointsAllowedAverage', inplace=True)
def_2016.sort_values('DraftKingsQuarterbackFantasyPointsAllowedAverage', inplace=True)
def_2017.sort_values('DraftKingsQuarterbackFantasyPointsAllowedAverage', inplace=True)

In [436]:
def_2017.reset_index(inplace=True)
def_2017['Rank'] = def_2017.index +1

In [437]:
#def_2015.index=def_2015['Team']

#def_2016.index=def_2016['Team']

def_2017.index = def_2017['Team']

In [58]:
qb_2015['year'] = 2015

In [68]:
qb_2016['year']=2016

In [78]:
qb_2017['year'] = 2017

In [438]:
# Map defensive rankings fron defense df to qb_df by yer and opponent

#qb_2015['def_rk'] = list(map(lambda x: def_2015.loc[x]['Rank'], qb_2015['Opponent']))
#qb_2016['def_rk'] = list(map(lambda x: def_2016.loc[x]['Rank'], qb_2016['Opponent']))
qb_2017['def_rk'] = list(map(lambda x: def_2017.loc[x]['Rank'], qb_2017['Opponent']))

C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [439]:
qb=pd.concat([qb_2015,qb_2016])

In [440]:
qb= pd.concat([qb, qb_2017])

In [441]:
qb.to_csv('input/qb/dk_stats_loc.csv')

In [73]:
qb_2016.head()

,PlayerID,Name,Team,Week,Opponent,PassingCompletions,PassingAttempts,PassingCompletionPercentage,PassingYards,PassingYardsPerAttempt,...,PassingInterceptions,PassingRating,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,FantasyPointsDraftKings,salary,year,def_rk
0,611,Joe Flacco,BAL,1,BUF,22,33,66.7,258,7.8,...,0,100.32,4,-3,-0.8,0,13.02,6700,2016,10
1,732,Matt Ryan,ATL,1,TB,27,39,69.2,334,8.6,...,0,112.55,3,10,3.3,0,27.36,7100,2016,17
2,2428,Carson Palmer,ARI,1,NE,24,37,64.9,271,7.3,...,0,104.67,2,1,0.5,0,18.94,7100,2016,6
3,2593,Aaron Rodgers,GB,1,JAX,20,34,58.8,199,5.9,...,0,95.10,4,16,4.0,1,23.56,8500,2016,8
4,3807,Ben Roethlisberger,PIT,1,WAS,27,37,73.0,300,8.1,...,1,112.44,1,-2,-2.0,0,25.80,7800,2016,27


In [80]:
qb_2017.head()

,PlayerID,Name,Team,Week,Opponent,PassingCompletions,PassingAttempts,PassingCompletionPercentage,PassingYards,PassingYardsPerAttempt,...,PassingInterceptions,PassingRating,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,FantasyPointsDraftKings,salary,year,def_rk
0,611,Joe Flacco,BAL,1,CIN,9,17,52.9,121,7.1,...,1,70.96,0,0,0.0,0,7.84,5500,2017,12
1,732,Matt Ryan,ATL,1,CHI,21,30,70.0,321,10.7,...,0,116.11,3,11,3.7,0,20.94,6900,2017,7
2,2428,Carson Palmer,ARI,1,DET,27,48,56.2,268,5.6,...,3,53.12,1,2,2.0,0,11.92,4300,2017,21
3,2593,Aaron Rodgers,GB,1,SEA,28,42,66.7,311,7.4,...,1,86.51,7,21,3.0,0,20.54,6000,2017,8
4,3807,Ben Roethlisberger,PIT,1,CLE,24,36,66.7,263,7.3,...,1,95.02,3,-8,-2.7,0,16.72,7000,2017,24


## Merge Dataframes

In [ ]:
data_qb = [qb_2015,qb_2016,qb_2017]

In [413]:
data_wr= [wr_2015,wr_2016,wr_2017]

In [82]:
qb= pd.concat(data)

In [414]:
wr= pd.concat(data_wr)

In [417]:
wr['ReceivingTargets'].max()

23

In [129]:
wr.shape

(5501, 23)

In [416]:
#qb.to_csv('input/qb/dk_stas_sal.csv', index=False)
wr.to_csv('input/wr/dk_stats_loc.csv', index=False)

In [ ]:
pd.read_

In [130]:
# add location to dataframes by mapping location of game from completed qb dataframe

qb = pd.read_csv('input/qb/dk_stats_loc.csv')

In [131]:
qb.head()

,Unnamed: 0,PlayerID,Name,Team,Week,Opponent,PassingCompletions,PassingAttempts,PassingCompletionPercentage,PassingYards,...,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,FantasyPointsDraftKings,salary,year,def_rk,h/a,surface
0,0,611,Joe Flacco,BAL,1,DEN,18,32,56.2,117,...,0,0,0.0,0,2.68,6700,2015,2,away,grass
1,1,732,Matt Ryan,ATL,1,PHI,23,34,67.6,298,...,5,7,1.4,0,18.62,7500,2015,30,home,turf
2,2,2428,Carson Palmer,ARI,1,NO,19,32,59.4,307,...,3,14,4.7,0,28.68,6500,2015,32,home,grass
3,3,2593,Aaron Rodgers,GB,1,CHI,18,23,78.3,189,...,8,35,4.4,0,23.06,8600,2015,15,away,grass
4,4,3807,Ben Roethlisberger,PIT,1,NE,26,38,68.4,351,...,0,0,0.0,0,22.04,7300,2015,13,away,turf


In [263]:
wr.head()

,PlayerID,Name,Team,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,ReceivingTouchdowns,...,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,year,salary,def_rk
0,204,Roddy White,ATL,1,PHI,8,4,50.0,84,0,...,0,0,0.0,0,0,0,12.4,2015,6500,30
1,1986,Jason Avant,KC,1,HOU,0,0,0.0,0,0,...,0,0,0.0,0,0,0,0.0,2015,3000,3
2,2429,Andre Johnson,IND,1,BUF,10,4,40.0,24,0,...,0,0,0.0,0,0,0,6.4,2015,6100,18
3,2950,Steve Johnson,LAC,1,DET,6,6,100.0,82,1,...,0,0,0.0,0,0,0,20.2,2015,3700,20
4,3330,Miles Austin,PHI,1,ATL,3,2,66.7,22,0,...,0,0,0.0,0,0,0,4.2,2015,3000,12


In [139]:
qb_2015 = qb[qb['year']==2015]
qb_2015.index = qb_2015['Team']
wr_2015.index = wr_2015['Team']

In [145]:
wr_2015.reset_index(inplace=True, drop =True)

In [148]:
location = pd.read_csv('input/location.csv')
location.drop('Unnamed: 0', axis=1,inplace=True)

In [143]:
wr_2015['h/a'] = home

In [401]:
temp = wr_2017

In [395]:
week1  = temp[temp['Week']==1]
week2 = temp[temp['Week']==2]
week3 = temp[temp['Week']==3]
week4 = temp[temp['Week']==4]
week5 = temp[temp['Week']==5]
week6 = temp[temp['Week']==6]
week7 = temp[temp['Week']==7]
week8 = temp[temp['Week']==8]
week9 = temp[temp['Week']==9]
week10 = temp[temp['Week']==10]
week11 = temp[temp['Week']==11]
week12 = temp[temp['Week']==12]
week13 = temp[temp['Week']==13]
week14 = temp[temp['Week']==14]
week15 = temp[temp['Week']==15]
week16 = temp[temp['Week']==16]
week17 = temp[temp['Week']==17]

In [393]:
loc_df = location[location['year']==2017]


In [402]:
week1.columns

Index(['PlayerID', 'Name', 'Team', 'Week', 'Opponent', 'ReceivingTargets',
       'Receptions', 'ReceptionPercentage', 'ReceivingYards',
       'ReceivingTouchdowns', 'ReceivingLong', 'ReceivingYardsPerTarget',
       'ReceivingYardsPerReception', 'RushingAttempts', 'RushingYards',
       'RushingYardsPerAttempt', 'RushingTouchdowns', 'Fumbles', 'FumblesLost',
       'FantasyPointsDraftKings', 'year', 'salary', 'def_rk', 'h/a'],
      dtype='object')

In [403]:
week2.columns

Index(['PlayerID', 'Name', 'Team', 'Week', 'Opponent', 'ReceivingTargets',
       'Receptions', 'ReceptionPercentage', 'ReceivingYards',
       'ReceivingTouchdowns', 'ReceivingLong', 'ReceivingYardsPerTarget',
       'ReceivingYardsPerReception', 'RushingAttempts', 'RushingYards',
       'RushingYardsPerAttempt', 'RushingTouchdowns', 'Fumbles', 'FumblesLost',
       'FantasyPointsDraftKings', 'year', 'salary', 'def_rk'],
      dtype='object')

In [434]:
loc_df[loc_df['week']==17]

home_list = []
for team in week17['Team']:
    if team in list(loc_df['home']):
        home_list.append('home')
    else:
        home_list.append('away')


week17['h/a'] = home_list


week17.head()

C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,PlayerID,Name,Team,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,ReceivingTouchdowns,...,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,year,salary,def_rk,h/a
2329,3601,Eric Weems,TEN,17,JAX,0,0,0.0,0,0,...,0,0.0,0,0,0,0.0,2017,3400,23,home
2330,5571,Larry Fitzgerald,ARI,17,SEA,13,8,61.5,55,0,...,0,0.0,0,0,0,13.5,2017,6600,5,home
2331,7651,Matthew Slater,NE,17,NYJ,0,0,0.0,0,0,...,0,0.0,0,0,0,0.0,2017,4100,24,away
2332,8373,Brandon Tate,BUF,17,MIA,0,0,0.0,0,0,...,0,0.0,0,0,0,0.0,2017,null,21,home
2333,8529,Mike Wallace,BAL,17,CIN,13,5,38.5,40,1,...,0,0.0,0,0,0,15.0,2017,3000,11,home


In [435]:
wr_2017 = pd.concat([wr_2017,week17])
wr_2017.tail()

,PlayerID,Name,Team,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,ReceivingTouchdowns,...,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,year,salary,def_rk,h/a
2476,19501,Justin Hardee,NO,17,TB,0,0,0.0,0,0,...,0,0.0,0,0,0,0.0,2017,null,20,away
2477,19514,Keelan Cole,JAX,17,TEN,8,4,50.0,33,0,...,0,0.0,0,1,1,6.3,2017,3200,15,away
2478,19539,Travis Rudolph,NYG,17,WAS,4,1,25.0,29,0,...,0,0.0,0,0,0,3.9,2017,3000,14,away
2479,19627,Chris Thompson,HOU,17,IND,3,3,100.0,54,0,...,0,0.0,0,0,0,8.4,2017,3500,26,home
2480,19668,Lance Lenoir,DAL,17,PHI,0,0,0.0,0,0,...,0,0.0,0,0,0,0.0,2017,6200,22,home


In [405]:
wr_2017 = pd.concat([week1,week2])
wr_2017.head()

,PlayerID,Name,Team,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,ReceivingTouchdowns,...,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,year,salary,def_rk,h/a
0,3601,Eric Weems,TEN,1,OAK,0,0,0.0,0,0,...,0,0.0,0,0,0,0.0,2017,3000,7,home
1,4556,Jordy Nelson,GB,1,SEA,8,7,87.5,79,1,...,0,0.0,0,0,0,20.9,2017,7600,5,away
2,5571,Larry Fitzgerald,ARI,1,DET,13,6,46.2,74,0,...,0,0.0,0,0,0,13.4,2017,5900,24,home
3,6767,Pierre Garcon,SF,1,CAR,10,6,60.0,81,0,...,0,0.0,0,0,0,14.1,2017,3000,17,away
4,8373,Brandon Tate,BUF,1,NYJ,0,0,0.0,0,0,...,0,0.0,0,0,0,0.0,2017,5300,24,home


In [217]:
wr_2015.shape

(2420, 24)

In [109]:
# Surface dictionary consisting of every team and the type of field they play on

surface_dict={'KC':'grass',
             'CAR':'grass',
              'WAS':'grass',
              'CLE':'grass',
              'MIA':'grass',
              'PIT':'grass',
              'GB':'grass',
              'SF':'grass',
              'PHI':'grass',
              'LAR':'grass',
              'BAL': 'grass',
              'TEN':'grass',
              'OAK':'grass',
              'LAC':'grass',
              'TB':'grass',
              'CHI':'grass',
              'DEN':'grass',
              'ARI':'grass',
              'DAL':'turf',
              'SEA':'turf',
              'DET':'turf',
              'ATL':'turf',
              'NE':'turf',
              'IND':'turf',
              'NO':'turf',
              'NYG':'turf',
              'NYJ':'turf',
              'HOU':'turf',
              'BUF':'turf',
              'CIN':'turf',
              'MIN':'turf',
              'JAX':'grass'
              
             }

In [387]:
wr_2016.index = wr_2016['Team']

In [436]:
# Add the tyep of field the player palyed on by mapping the field on which the game was played to the surface dictionary

surface = []
for idx, team in enumerate(wr_2017['Team']):
    if wr_2017.iloc[idx]['h/a']=='home':
        surface.append(surface_dict[team])
    else:
        surface.append(surface_dict[wr_2017.iloc[idx]['Opponent']])


wr_2017['surface'] = surface

In [437]:
wr_2017.head() 

,PlayerID,Name,Team,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,ReceivingTouchdowns,...,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,year,salary,def_rk,h/a,surface
0,3601,Eric Weems,TEN,1,OAK,0,0,0.0,0,0,...,0.0,0,0,0,0.0,2017,3000,7,home,grass
1,4556,Jordy Nelson,GB,1,SEA,8,7,87.5,79,1,...,0.0,0,0,0,20.9,2017,7600,5,away,turf
2,5571,Larry Fitzgerald,ARI,1,DET,13,6,46.2,74,0,...,0.0,0,0,0,13.4,2017,5900,24,home,grass
3,6767,Pierre Garcon,SF,1,CAR,10,6,60.0,81,0,...,0.0,0,0,0,14.1,2017,3000,17,away,grass
4,8373,Brandon Tate,BUF,1,NYJ,0,0,0.0,0,0,...,0.0,0,0,0,0.0,2017,5300,24,home,turf


In [438]:
wr = pd.concat([wr,wr_2017])

In [439]:

wr.tail()

,PlayerID,Name,Team,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,ReceivingTouchdowns,...,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsDraftKings,year,salary,def_rk,h/a,surface
2476,19501,Justin Hardee,NO,17,TB,0,0,0.0,0,0,...,0.0,0,0,0,0.0,2017,null,20,away,grass
2477,19514,Keelan Cole,JAX,17,TEN,8,4,50.0,33,0,...,0.0,0,1,1,6.3,2017,3200,15,away,grass
2478,19539,Travis Rudolph,NYG,17,WAS,4,1,25.0,29,0,...,0.0,0,0,0,3.9,2017,3000,14,away,grass
2479,19627,Chris Thompson,HOU,17,IND,3,3,100.0,54,0,...,0.0,0,0,0,8.4,2017,3500,26,home,turf
2480,19668,Lance Lenoir,DAL,17,PHI,0,0,0.0,0,0,...,0.0,0,0,0,0.0,2017,6200,22,home,turf


In [441]:
wr.to_csv('input/wr/dk_stats_loc.csv', index=False)